# Non linear feature extraction of the stratosphere

## Set-up: 

### Imports:

In [7]:
from scipy.spatial import distance_matrix
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
from time import sleep
import warnings
from sklearn.neighbors import NearestNeighbors
import scipy.sparse.linalg as linalg
import scipy.signal as signal
import seaborn as sns
import matplotlib.dates as mdates
from datetime import datetime
import os
import math 
warnings.filterwarnings('ignore')

from functions_takens import *

%load_ext jupyternotify
%load_ext autoreload
%autoreload 2

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Constants:

In [2]:
# Percentage of nearest neighbours:
PERC_NEIGH = 10
print(f'Percentage of nearest neighbours: {PERC_NEIGH/100}')

# Number of eigenmaps to compute:
NUM_EIGENVALUES = 21
print(f'Number of eigenmaps: {NUM_EIGENVALUES}')

# Data set to consider: ('raw/anomalies')
DATA = 'anomalies'
print(f'Data set considered: {DATA}')

# Path to input data:
INPUT_DATA = '../../../data/vandermeer/input_data/'
print(f'Path to input data: {INPUT_DATA}')

# Wether to do NLSA or Laplacian Eigenmaps. Takens True for NLSA
USE_TAKENS = True
print(f'Using takens embedding: {USE_TAKENS}')

# Time-step in Takens embedding:
TAU = 4 * 30 * 2
print(f'tau = {TAU/(4*30)} months')

BEGIN_YEAR = 1979
END_YEAR = 2019


Percentage of nearest neighbours: 0.1
Number of eigenmaps: 21
Data set considered: raw
Path to input data: ../../../data/vandermeer/input_data/
Using takens embedding: True
tau = 2.0 months


Constants for plots:

In [3]:
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 14

plt.rc('font', size=MEDIUM_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

### Paths:

Global path to save data: 

In [4]:
if DATA == 'raw':
    PATH = '../../../data/vandermeer/pickles/raw/'
elif DATA == 'anomalies': 
    PATH = '../../../data/vandermeer/pickles/anomalies/'
if not os.path.exists(PATH):
        os.makedirs(PATH)
print(f'Global path: {PATH}')

Global path: ../../../data/vandermeer/pickles/raw/


Path to save data according to the percentage of neighbours used: 

In [5]:
PATH1 = PATH + str(PERC_NEIGH)+'perc/'
if not os.path.exists(PATH1):
        os.makedirs(PATH1)
print(f'Precise path: {PATH1}')

Precise path: ../../../data/vandermeer/pickles/raw/10perc/


Path to save data of simple kernel (binary kernel): 

In [6]:
# path to simple_kernel:
PATH_SIMPLE = PATH1 + 'simple_kernel/'
if not os.path.exists(PATH_SIMPLE):
        os.makedirs(PATH_SIMPLE)
print(f'Path to simple kernel: {PATH_SIMPLE}')

PATH_SIMPLE_TAKENS = PATH1 + 'simple_kernel/takens/'
if not os.path.exists(PATH_SIMPLE_TAKENS):
        os.makedirs(PATH_SIMPLE_TAKENS)
print(f'Path to simple kernel for NLSA: {PATH_SIMPLE_TAKENS}')

Path to simple kernel: ../../../data/vandermeer/pickles/raw/10perc/simple_kernel/
Path to simple kernel for NLSA: ../../../data/vandermeer/pickles/raw/10perc/simple_kernel/takens/


Path to save results of NLSA with correct neighbour percentage:

In [ ]:
# path to NLSA results:
PATH_TAKENS = PATH + str(PERC_NEIGH)+'perc/takens/'
if not os.path.exists(PATH_TAKENS):
        os.makedirs(PATH_TAKENS)
print(f'Path to NLSA results: {PATH_TAKENS}')

### Load data:

Load inut raw or anomalies data with basis

In [ ]:
%%time
anomalies_cf = pd.read_csv(INPUT_DATA + 'anomalies_coefficients.csv', sep=',')
raw_cf = pd.read_csv(INPUT_DATA + 'raw_data_coefficients.csv', sep=',')
basis_cf = pd.read_csv(INPUT_DATA + 'basis_functions.csv', sep=',')

if DATA == 'raw':
    df = raw_cf
elif DATA == 'anomalies':
    df = anomalies_cf

# remove useless axes:
df = df.drop(['Unnamed: 0', 'Date'], axis=1)
print('Sample of data:')
print(f'Shape of {DATA} data: {df.shape}')
pd.DataFrame(df).head(3)

## Laplacian Eigenmaps:

Laplacian eigenmap decomposition according to this [paper](https://www2.imm.dtu.dk/projects/manifold/Papers/Laplacian.pdf).

### Distance matrix:
Calculate matrix of distances between all points:

In [ ]:
# Run this to re-compute distance matrix:
"""
%%notify
print(f'Distance matrix for {DATA}')
distance_df = pd.DataFrame(distance_matrix(df.values, df.values),
                           index=df.index,
                           columns=df.index)
distance_df.to_pickle(PATH + 'distance_matrix.pkl')
"""

Load pre-computed distance matrix:

In [ ]:
%%time
print(f'Distance matrix read from {PATH1}')
distance_matrix = pd.read_pickle(PATH1+'distance_matrix.pkl').values

#check distance matrix is symmetric:
assert(np.all(distance_matrix.T == distance_matrix))

print(f'Distance matrix shape: {distance_matrix.shape}')
print('Sample of distance matrix:')
pd.DataFrame(distance_matrix).head(3)

Image of distance matrix:

In [ ]:
%%time
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(distance_matrix)

### Taken's embedding:

Calculate new distance matrix from the original one using Taken's embedding. In this new matrix 

$dist(Y(t_1), Y(t_2))=\sqrt(\sum_{j=0}^{\tau-1}dist(X(t_1 +j),X(t_2+j))^2)$ 

Where $\tau$ is the time-step.

Distance matrix:

In [ ]:
%%time
distance_matrix = pd.read_pickle(PATH1 + 'distance_matrix.pkl').values
print(f'Distance matrix shape: {distance_matrix.shape}')

In [ ]:
if DATA == 'raw':
    df = pd.read_csv(INPUT_DATA + 'raw_data_coefficients.csv', sep=',')
elif DATA == 'anomalies': 
    df = pd.read_csv(INPUT_DATA + 'anomalies_coefficients.csv', sep=',')
print(f'Input data shape: {df.shape}')

Get time information and the number of measures per year: 

In [ ]:
time = df['Date']
time = pd.to_datetime(time)
df_ = df.drop(['Unnamed: 0', 'Date'], axis=1)
df_time = pd.concat([time, df_], axis=1)
df_time = df_time.set_index('Date')

years = range(BEGIN_YEAR, END_YEAR)
years = [str(y) for y in years]

counts = {}
sum_ = 0
for y in years:
    sum_ += len(df_time[y])
    counts[y] = len(df_time[y])

Get positions of last measure per year so that we don't cross to another year when computing the Takens matrix:

In [ ]:
cs = [counts[i] for i in years]
last_year_pos = [np.sum(cs[:i+1]) for i in range(len(cs))]

entire_winters = END_YEAR-BEGIN_YEAR-1
print(f'Number of entire winters: {entire_winters}')

Get last and first positions of entire winters:

In [ ]:
last_pos_winters = give_last_post_winters(last_year_pos, years, df_time)
first_pos_winters = give_first_post_winters(last_year_pos, years, df_time)
assert(len(last_pos_winters)==len(first_pos_winters))

np.save(PATH1+'last_pos_winters.npy', last_pos_winters)
np.save(PATH1+'first_pos_winters.npy', first_pos_winters)

In [ ]:
indices_of_points = []
for i in range(len(first_pos_winters)):
    indices_of_points.append(
        range(first_pos_winters[i] + TAU, last_pos_winters[i] + 1, 1))
for i in indices_of_points:
    assert (i[-1] - i[0] > 600)

indices_of_points

In [ ]:
%%time
# Safety check to see everything is working all right:
# total number of points:
le = np.sum([len(i) for i in indices_of_points])
# time series corresponding to those points:
time_nlsa = time[indices_of_points[0]]
for i in range(1, len(last_pos_winters)):
    time_nlsa = pd.concat([time_nlsa, time[indices_of_points[i]]], axis = 0)
assert(le==len(time_nlsa))

Unwrap the indices of all ranges of `indices_of_points` to get one list of all indices we need to go over. 

In [ ]:
indices_m = []
for j in range(len(indices_of_points)):
    for i in range(len(indices_of_points[j])):
        indices_m.append(indices_of_points[j][i])
assert(len(indices_m)==le)
print(f'First and last indice: {indices_m[0]}, {indices_m[-1]}')

Compute the new distance matrix:

In [ ]:
%%time
#Number of entire winters:
num_years = END_YEAR-BEGIN_YEAR-1
print(f'Number of years: {num_years}')

m,n  = le, le
dist_Y = np.zeros((m,n))
print(f'New distance matrix shape: {dist_Y.shape}')

In [ ]:
"""%%time
dist_Y = apply_takens(dist_Y, distance_matrix, indices_m, TAU, PATH1)"""

Right now just upper triangle matrix so make it symmetric:

In [ ]:
%%time
print(f'Reading upper triangle matrix at {PATH1}:')
dist_Y = pd.read_pickle(PATH1+'dist_Y_takens_final.pkl').values

# Make matrix symetric:
takens_matrix = dist_Y + dist_Y.T

# Save NLSA distance matrix matrix:
print(f'Save NLSA distance matrix at {PATH1}')
takens_df = pd.DataFrame(takens_matrix)
takens_df.to_pickle(PATH1 + 'distance_matrix_takens_final.pkl')

del dist_Y

Read NLSA distance matrix from memory:

In [ ]:
PATH1

In [ ]:
%%time
print('Reading Takens matrix:')
takens_matrix = pd.read_pickle(PATH1 + 'distance_matrix_takens.pkl').values

#check distance matrix is symmetric:
assert(np.all(takens_matrix.T == takens_matrix))

In [ ]:
takens_matrix.shape

In [ ]:
pd.DataFrame(takens_matrix)

In [ ]:
%%time
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(takens_matrix)
del takens_matrix

### Nearest neighbours:
Calculate matrix with neighbouring vertices, e.g. 1 for $x_j,x_i$ if $x_j$ or $x_i$ selected the other as a closest  neighbour and 0 otherwise. This weight matrix will also serve as the simple kernel. Creates the binary weight matrix.  

In [ ]:
%%time
# Percentage of neighbours
perc = PERC_NEIGH / 100
print(f'Starting nearest neighbours for {perc*100}% nearest neihbours ')

# Reading input data:
if DATA == 'raw':
    print(f'Reading raw input data from: {INPUT_DATA}')
    df = pd.read_csv(INPUT_DATA + 'raw_data_coefficients.csv', sep=',')
elif DATA == 'anomalies': 
    print(f'Reading anomalies input data from: {INPUT_DATA}')
    df = pd.read_csv(INPUT_DATA + 'anomalies_coefficients.csv', sep=',')

print(f'Reading distance matrix')

if USE_TAKENS == True:
    print(f'Using takens embedding, reading from {PATH_SIMPLE}')
    distance_matrix = pd.read_pickle(PATH1 + 'distance_matrix_takens.pkl').values
    print(f'Distance matrix shape: {distance_matrix.shape}')
else:
    print(f'Using normal distance matrix, reading from {PATH1}')
    distance_matrix = pd.read_pickle(PATH1+'distance_matrix.pkl').values
    print(f'Distance matrix shape: {distance_matrix.shape}')

# K-nearest neighbours:
print(f'Look for {perc*100}% nearest neihbours:')
K = int(len(df.values) * perc)
N = len(distance_matrix)
weight_matrix = np.zeros((N, N))

for i in tqdm(range(N)):
    # select K closest neihbours:
    indices = np.argsort(distance_matrix[i])[:K]
    for j in indices:
        if i != j and weight_matrix[i, j] == 0:
            weight_matrix[i, j] += 1
weight_matrix = weight_matrix.T

# Make weight matrix is symmetric:
for i in tqdm(range(N)):
    indices = np.argsort(distance_matrix.T[i])[:K]
    for j in indices:
        if i != j and weight_matrix[i, j] == 0:
            weight_matrix[i, j] += 1
weight_df = pd.DataFrame(weight_matrix)

# Save weight matrix:
if USE_TAKENS:
    PATH_SAVE = PATH_SIMPLE_TAKENS+'weight_matrix_takens_{}.pkl'.format(PERC_NEIGH)
    print('Saving weight matrix at {}'.format(PATH_SAVE))
    weight_df.to_pickle(PATH_SAVE)
else:
    PATH_SAVE = PATH_SIMPLE+'weight_matrix_{}.pkl'.format(PERC_NEIGH)
    print('Saving weight matrix at {}'.format(PATH_SAVE))
    weight_df.to_pickle(PATH_SAVE)

del weight_df, weight_matrix, distance_matrix
print('Done')

In [ ]:
%%time
# Read weight matrix:
if USE_TAKENS:
    print(f'Reading binary weight matrix from {PATH_SIMPLE_TAKENS}')
    weight_m = pd.read_pickle(PATH_SIMPLE_TAKENS +
                          'weight_matrix_takens_{}.pkl'.format(PERC_NEIGH)).values
else:
    print(f'Reading binary weight matrix from {PATH_SIMPLE}')
    weight_m = pd.read_pickle(PATH_SIMPLE +
                          'weight_matrix_{}.pkl'.format(PERC_NEIGH)).values
# Test if matrix is symetric:
assert (np.all(weight_m.T == weight_m))
print(f'Weight matrix shape: {weight_m.shape}')

In [ ]:
pd.DataFrame(weight_m).head(5)

Look at image of weight matrix:

In [ ]:
%%time
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(weight_m)
del weight_m

### Binary kernel: 

Compute Laplacian eigenmaps for the binary kernel (0/1). 

#### Diagonal matrix:
Diagonal matrix from binary weight matrix, computed as $D_{ij} = \sum_{j}W_{ij}$

In [ ]:
%%time

if USE_TAKENS:
    weight_m = pd.read_pickle(PATH_SIMPLE_TAKENS +
                          'weight_matrix_takens_{}.pkl'.format(PERC_NEIGH)).values
else:
    weight_m = pd.read_pickle(PATH_SIMPLE +
                          'weight_matrix_{}.pkl'.format(PERC_NEIGH)).values

# Compute diagonal matrix:
D = np.zeros((len(weight_m), len(weight_m)))
print('Computing diagonal matrix:')
for i in tqdm(range(len(weight_m))):
    D[i, i] = np.sum(weight_m[i])
D_df = pd.DataFrame(D)


if USE_TAKENS:
    print(f'Saving diagonal matrix at: {PATH_SIMPLE_TAKENS}')
    D_df.to_pickle(PATH_SIMPLE_TAKENS+'diagonal_matrix_takens_{}.pkl'.format(PERC_NEIGH))
else:
    print(f'Saving diagonal matrix at: {PATH_SIMPLE}')
    D_df.to_pickle(PATH_SIMPLE+'diagonal_matrix_{}.pkl'.format(PERC_NEIGH))
print('Done!')

#### Laplacian matrix:

Laplacian matrix computed as $L = D-W$

In [ ]:
%%time
print('Computing Laplacian matrix:')

L = np.subtract(D, weight_m)
L_df = pd.DataFrame(L)

if USE_TAKENS: 
    print(f'Saving laplacian matrix at: {PATH_SIMPLE_TAKENS}')
    L_df.to_pickle(PATH_SIMPLE_TAKENS + 'laplacian_simple_matrix_takens_{}.pkl'.format(PERC_NEIGH))
else:
    print(f'Saving laplacian matrix at: {PATH_SIMPLE}')
    L_df.to_pickle(PATH_SIMPLE + 'laplacian_simple_matrix_{}.pkl'.format(PERC_NEIGH))

del L, L_df, D, weight_m
print('Done!')

#### Eigenvalues:
Eigenvalues and eigenvectors for binary kernel. 

In [ ]:
%%time

if USE_TAKENS:
    print(f'Reading D and L from: {PATH_SIMPLE_TAKENS}')
    D = pd.read_pickle(
        PATH_SIMPLE_TAKENS +
        'diagonal_matrix_takens_{}.pkl'.format(PERC_NEIGH)).values
    print(f'Diagonal matrix shape: {D.shape}')
    L = pd.read_pickle(
        PATH_SIMPLE_TAKENS +
        'laplacian_simple_matrix_takens_{}.pkl'.format(PERC_NEIGH)).values
    print(f'Laplacian matrix shape: {L.shape}')

else:
    print(f'Reading D and L from: {PATH_SIMPLE}')
    D = pd.read_pickle(PATH_SIMPLE +
                       'diagonal_matrix_{}.pkl'.format(PERC_NEIGH)).values
    L = pd.read_pickle(
        PATH_SIMPLE +
        'laplacian_simple_matrix_{}.pkl'.format(PERC_NEIGH)).values

# Compute first eigenvalues
print(f'Computing {NUM_EIGENVALUES} eigenvalues:')
w, eigv = linalg.eigs(L, k=NUM_EIGENVALUES, M=D, which='SM')

# Save values:
if USE_TAKENS:
    print(f'Saving eigenvalues and eigenvectors to: {PATH_SIMPLE_TAKENS}')
    pd.DataFrame(w).to_pickle(PATH_SIMPLE_TAKENS +
                              'eigenvalues_takens_{}.pkl'.format(PERC_NEIGH))
    pd.DataFrame(eigv).to_pickle(
        PATH_SIMPLE_TAKENS + 'eigenvectors_takens_{}.pkl'.format(PERC_NEIGH))
    print(f'Eigenvector shape: {eigv.shape}')
else:
    print(f'Saving eigenvalues and eigenvectors to: {PATH_SIMPLE}')
    pd.DataFrame(w).to_pickle(PATH_SIMPLE +
                              'eigenvalues_{}.pkl'.format(PERC_NEIGH))
    pd.DataFrame(eigv).to_pickle(PATH_SIMPLE +
                                 'eigenvectors_{}.pkl'.format(PERC_NEIGH))
del L, D, w, eigv
print('Done!')

### Heat kernel: 
The heat kernel is computed as $W_{ij} = exp(\frac{-||x_j-x_i||^2_2}{t})$ for connected neighbours and 0 otherwise.
Source:[paper](https://www2.imm.dtu.dk/projects/manifold/Papers/Laplacian.pdf)

#### Choice of bandwidth: 
Compute the right bandwidth $t$ values for the heat kernel. Try the mean, median and maximum over distances of neihgbouring vertices.

In [ ]:
%%time
print(f'Reading distance and weight matrix from: {PATH1}')

if USE_TAKENS: 
    print('Using takens:')
    distance_matrix = pd.read_pickle(PATH1 + 'distance_matrix_takens.pkl').values
    weight_m = pd.read_pickle(PATH_SIMPLE_TAKENS +'weight_matrix_takens_{}.pkl'.format(PERC_NEIGH)).values
else:
    distance_matrix = pd.read_pickle(PATH1+'distance_matrix.pkl').values
    weight_m = pd.read_pickle(PATH_SIMPLE +'weight_matrix_{}.pkl'.format(PERC_NEIGH)).values

In [ ]:
%%time
# Select only distances that are chosen in the nearest neighbours step:
mult_df = np.multiply(distance_matrix, weight_m)
non_zero_mult = np.extract(mult_df > 0, mult_df)

In [ ]:
# Calculate the mean, median and max over non-zero distances:
mean_distances = np.mean(non_zero_mult)
median_dist = np.median(non_zero_mult)
max_distances = np.max(non_zero_mult)

t = [mean_distances, median_dist, max_distances]

ts = {
    'mean_distances': mean_distances,
    'median_dist': median_dist,
    'max_distances': max_distances,
    'mean_dist_all': np.mean(distance_matrix),
    'max_dist_all': np.max(distance_matrix),
}
if USE_TAKENS:
    np.save(PATH1 + 't_takens_.npy', np.array(t))
else:
    np.save(PATH1+'t.npy', np.array(t))
print(ts)

In [ ]:
fig = plt.figure(figsize = (8,5))
axs = plt.subplot(1,1,1)
axs.hist(non_zero_mult)
#plt.title('Non zero distances between neighbours:')
axs.axvline(t[0], color = 'green', label = 'mean')
axs.axvline(t[1], color = 'orange', label = 'median')
axs.axvline(t[2], color = 'red', label = 'max')
axs.set_xlabel('Distance between neihbours')
axs.set_ylabel('Count')
plt.legend()

In [ ]:
del distance_matrix, weight_m, mult_df, non_zero_mult

#### Heat matrix: 

Computing heat kernels:

In [ ]:
%%time

if USE_TAKENS:
    t = np.load(PATH1+'t_takens_.npy')
    chosen_t = t[0]
    corr_t = ['mean']
    
    print(f'Reading distance and weight matrix from: {PATH_SIMPLE}')
    distance_matrix = pd.read_pickle(PATH1 + 'distance_matrix_takens.pkl').values
    weight_m = pd.read_pickle(PATH_SIMPLE_TAKENS+'weight_matrix_takens_{}.pkl'.format(PERC_NEIGH)).values
    
    print('Computing heat matrix for bandwitdh {}: {}'.format(corr_t[0],chosen_t))
    PATH2 = PATH1+'t_'+corr_t[0]+'/takens/'
    if not os.path.exists(PATH2):
            os.makedirs(PATH2)

    distance_df = pd.DataFrame(distance_matrix)
    # Create heat matrix:
    heat_matrix_df = distance_df.apply(lambda x: np.exp(-(x**2) / (chosen_t**2)))
    heat_matrix_df = pd.DataFrame(np.multiply(weight_m, heat_matrix_df))
    print(f'Saving heat matrix at {PATH2}')
    heat_matrix_df.to_pickle(PATH2+'heat_matrix_'+'t_'+corr_t[0]+ '_takens_.pkl')
    
else:
    t = np.load(PATH1+'t.npy')
    corr_t = ['mean', 'med', 'max']
    
    print(f'Reading distance and weight matrix from: {PATH1}')
    distance_matrix = pd.read_pickle(PATH1+'distance_matrix.pkl').values
    weight_m = pd.read_pickle(PATH_SIMPLE+'weight_matrix_{}.pkl'.format(PERC_NEIGH)).values
    
    for i in range(3):
        chosen_t = t[i]
        print('Computing heat matrix for bandwitdh {}: {}'.format(corr_t[i],chosen_t))

        PATH2 = PATH1+'t_'+corr_t[i]+'/'
        if not os.path.exists(PATH2):
            os.makedirs(PATH2)

        distance_df = pd.DataFrame(distance_matrix)
        # Create heat matrix:
        heat_matrix_df = distance_df.apply(lambda x: np.exp(-(x**2) / (chosen_t**2)))
        heat_matrix_df = pd.DataFrame(np.multiply(weight_m, heat_matrix_df))
        heat_matrix_df.to_pickle(PATH2+'heat_matrix_'+'t_'+corr_t[i]+ '_.pkl')
del heat_matrix_df, distance_matrix, weight_m, distance_df

Load pre-computed heat kernels for 10% neighbours:

In [ ]:
%%time
corr_t = ['mean', 'med', 'max']


if USE_TAKENS:
    PATH_READ = '../../../data/vandermeer/pickles/{}/10perc/'.format(DATA)
    print(f'Path to heat matrices:{PATH_READ}')
    print(f'Reading heat matrix for t = {corr_t[0]}:')
    heat_matrix_mean = pd.read_pickle(PATH_READ +'t_'+corr_t[0]+ '/takens/' +
                                      'heat_matrix_' +'t_'+corr_t[0]+'_takens_.pkl').values
    print(f'Heat matrix shape: {heat_matrix_mean.shape}')
else:
    PATH_READ = '../../../data/vandermeer/pickles/{}/10perc/'.format(DATA)
    print(f'Path to heat matrices:{PATH_READ}')
    print(f'Reading heat matrix for t = {corr_t[0]}:')
    heat_matrix_mean = pd.read_pickle(PATH_READ +'t_'+corr_t[0]+ '/' +
                                      'heat_matrix_' +'t_'+corr_t[0]+'_.pkl').values
    print(f'Reading heat matrix for t = {corr_t[1]}:')
    heat_matrix_max = pd.read_pickle(PATH_READ +'t_'+corr_t[1]+ '/' +
                                     'heat_matrix_' +'t_'+corr_t[1]+'_.pkl').values
    print(f'Reading heat matrix for t = {corr_t[2]}:')
    heat_matrix_med = pd.read_pickle(PATH_READ +'t_'+corr_t[2]+ '/' +
                                     'heat_matrix_' +'t_'+corr_t[2]+'_.pkl').values

Load pre-computed heat kernels for 20% neighbours if you want to compare to those:

In [ ]:
%%time
if not USE_TAKENS:
    corr_t = ['mean', 'med', 'max']
    PATH_READ = '../../../data/vandermeer/pickles/{}/20perc/'.format(DATA)
    print(f'Path to heat matrices:{PATH_READ}')
    print(f'Reading heat matrix for t = {corr_t[0]}:')
    heat_matrix_mean2 = pd.read_pickle(PATH_READ +'t_'+corr_t[0]+ '/' +
                                      'heat_matrix_' +'t_'+corr_t[0]+
                                      '_.pkl').values
    print(f'Reading heat matrix for t = {corr_t[1]}:')
    heat_matrix_max2 = pd.read_pickle(PATH_READ +'t_'+corr_t[1]+ '/' +
                                     'heat_matrix_' +'t_'+corr_t[1]+
                                     '_.pkl').values
    print(f'Reading heat matrix for t = {corr_t[2]}:')
    heat_matrix_med2 = pd.read_pickle(PATH_READ +'t_'+corr_t[2]+ '/' +
                                     'heat_matrix_' +'t_'+corr_t[2]+
                                     '_.pkl').values

Plot sample of heat kernels as image:

In [ ]:
if not USE_TAKENS:
    fig, axs = plt.subplots(1, 3, figsize=(10, 10))
    i = 0
    corr_t = ['mean', 'med', 'max']
    matrices = [heat_matrix_mean[:10, :10],heat_matrix_max[:10, :10],heat_matrix_med[:10, :10]]
    for chosen_t in corr_t:
        axs[i].imshow(matrices[i])
        axs[i].set_title(chosen_t)
        i += 1

Plot histogram of non zero values of heat kernels:

In [ ]:
%%time
if USE_TAKENS:
    fig, axs = plt.subplots(1,1, figsize = (5,5))
    corr_t = ['mean']
    non_zero_heat = np.extract(heat_matrix_mean>0, heat_matrix_mean)
    axs.set_title('Heat kernel, t = {} distance'.format(corr_t[0]))
    axs.hist(non_zero_heat, label = '10 perc')

In [ ]:
%%time

if not USE_TAKENS:
    fig, axs = plt.subplots(1,3, figsize = (15,5))
    i = 0
    corr_t = ['mean', 'med', 'max']
    matrices = [heat_matrix_mean, heat_matrix_max, heat_matrix_med]
    for i in range(3):
        heat_matrix = matrices[i]
        non_zero_heat = np.extract(heat_matrix>0, heat_matrix)
        axs[i].set_title('Heat kernel, t = {} distance'.format(corr_t[i]))
        axs[i].hist(non_zero_heat, label = '10 perc')
        i+=1

Plot overlapping histogram of non zero values of heat kernels for 10% and 20%:

In [ ]:
%%time
if not USE_TAKENS:
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    i = 0
    matrices = [heat_matrix_mean, heat_matrix_max, heat_matrix_med]
    matrices2 = [heat_matrix_mean2, heat_matrix_max2, heat_matrix_med2]
    corr_t = ['mean', 'med', 'max']
    for i in range(3):
        heat_matrix = matrices[i]
        heat_matrix2 = matrices2[i]
        non_zero_heat = np.extract(heat_matrix > 0, heat_matrix)
        non_zero_heat2 = np.extract(heat_matrix2 > 0, heat_matrix2)
        t = ['mean', 'max', 'median']
        axs[i].set_title('Heat kernel, t = {} distance'.format(corr_t[i]))
        axs[i].hist(non_zero_heat, label='10 perc', alpha=0.5)
        axs[i].hist(non_zero_heat2, label='20 perc', alpha=0.5)
        i += 1
    plt.legend()
    del matrices, matrices2, heat_matrix_mean, heat_matrix_max, heat_matrix_med, heat_matrix_mean2, heat_matrix_max2, heat_matrix_med2

#### Diagonal weight matrix: 
Compute diagonal matrix as $D_{ii} = \sum_j W_{ij}$

In [ ]:
if USE_TAKENS:
    corr_t = ['mean']
    print(f'Computing diagonal matrix for heat kernel with t: {corr_t[0]}')
    PATH2 = PATH1+'t_'+corr_t[0]+'/takens/'
    if not os.path.exists(PATH2):
        os.makedirs(PATH2)
    wm = pd.read_pickle(PATH1+'t_'+corr_t[0]+'/takens/heat_matrix_t_'+corr_t[0]+'_takens_.pkl').values
    D = np.zeros((len(wm),len(wm)))
    for i in tqdm(range(len(wm))):
        D[i,i] = np.sum(wm[i])
    D_df = pd.DataFrame(D)
    print(f'Writing diagonal matrix to {PATH2}')
    D_df.to_pickle(PATH2+'diagonal_heat_matrix_t_'+corr_t[0]+'_takens_.pkl')
else:
    corr_t = ['mean', 'med', 'max']
    for j in range(3):
        print(f'Computing diagonal matrix for heat kernel with t: {corr_t[j]}')
        PATH2 = PATH1+'t_'+corr_t[j]+'/'
        if not os.path.exists(PATH2):
            os.makedirs(PATH2)
        wm = pd.read_pickle(PATH1+'t_'+corr_t[j]+'/heat_matrix_t_'+corr_t[j]+'_.pkl').values
        D = np.zeros((len(wm),len(wm)))
        for i in tqdm(range(len(wm))):
            D[i,i] = np.sum(wm[i])
        D_df = pd.DataFrame(D)
        print(f'Writing diagonal matrix to {PATH2}')
        D_df.to_pickle(PATH2+'diagonal_heat_matrix_t_'+corr_t[j]+'_.pkl')
del D_df, D, wm

Load pre-computed diagonal matrix_:

In [ ]:
%%time
if USE_TAKENS:
    D = pd.read_pickle(PATH1+'t_mean/takens/diagonal_heat_matrix_'+'t_mean_takens_.pkl').values
else:
    D = pd.read_pickle(PATH1+'t_mean/diagonal_heat_matrix_'+'t_mean_.pkl').values
print(f'Shape of diagonal matrix: {D.shape}')
pd.DataFrame(D).head(5)

In [ ]:
del D

#### Laplacian matrix: 

Compute Laplacian matrix as $L = D- W$

In [ ]:
%%time
if USE_TAKENS:
    corr_t = ['mean']
    print(f'Computing Laplacian matrix for heat kernel with t: {corr_t[0]}')
    PATH2 = PATH1 + 't_' + corr_t[0] + '/takens/'
    if not os.path.exists(PATH2):
        os.makedirs(PATH2)
    print('Reading heat matrix:')
    wm = pd.read_pickle(PATH2+'heat_matrix_t_'+corr_t[0]+'_takens_.pkl').values
    print('Reading diagonal matrix:')
    D = pd.read_pickle(PATH2+'diagonal_heat_matrix_t_'+corr_t[0]+'_takens_.pkl').values
    print('Calculating Laplacian:')
    L = np.subtract(D, wm)
    L_df = pd.DataFrame(L)
    print(f'Writing Laplacian to {PATH2}')
    L_df.to_pickle(PATH2+'laplacian_heat_matrix_t_'+corr_t[0]+'_takens_.pkl')

else:
    corr_t = ['mean', 'med', 'max']
    for j in range(3):
        print(f'Computing Laplacian matrix for heat kernel with t: {corr_t[j]}')
        PATH2 = PATH1 + 't_' + corr_t[j] + '/'
        if not os.path.exists(PATH2):
            os.makedirs(PATH2)
        print('Reading heat matrix:')
        wm = pd.read_pickle(PATH2+'heat_matrix_t_'+corr_t[j]+'_.pkl').values
        print('Reading diagonal matrix:')
        D = pd.read_pickle(PATH2+'diagonal_heat_matrix_t_'+corr_t[j]+'_.pkl').values
        print('Calculating Laplacian:')
        L = np.subtract(D, wm)
        L_df = pd.DataFrame(L)
        L_df.to_pickle(PATH2+'laplacian_heat_matrix_t_'+corr_t[j]+'_.pkl')
    del L_df, L, D, wm
print('Done!')

Load pre-computed laplacian matrix:

In [ ]:
%%time

if USE_TAKENS:
    D = pd.read_pickle(PATH1+'t_mean/takens/laplacian_heat_matrix_t_mean_takens_.pkl').values
else:
    D = pd.read_pickle(PATH1+'t_mean/laplacian_heat_matrix_t_mean_.pkl').values
print(f'Shape of Laplacian matrix: {L.shape}')
pd.DataFrame(L).head(5)

In [ ]:
del L

#### Eigenvalues: 

Eigendecomposition of: $Lf = \gamma Df$ where $f$ are the eigenvector solutions ordered according to their increasing eigenvalue $\lambda_0 = 0 < \lambda_1 < ...$

Computing eigendecomposition:

In [ ]:
%%time
chosen_t = 'mean'
print(f'Computing eigenvalues for heat kernel with t={chosen_t}')

if not os.path.exists(PATH2):os.makedirs(PATH2)

if USE_TAKENS:  
    PATH2 = PATH1+'t_{}/takens/'.format(chosen_t)
    print(f'Loading L from:{PATH2}')   
    L = pd.read_pickle(PATH2+'laplacian_heat_matrix_t_'+chosen_t+'_takens_.pkl').values

    print(f'Loading D from:{PATH2}')   
    D = pd.read_pickle(PATH2+'diagonal_heat_matrix_t_'+chosen_t+'_takens_.pkl').values

    print(f'Computing {NUM_EIGENVALUES} eigenvalues:')
    w, eigv = linalg.eigs(L, k=NUM_EIGENVALUES, M=D, which='SM')

    print('Saving eigendecomposition:')
    pd.DataFrame(w).to_pickle(PATH2 + 'eigenvalues_heat_matrix_t_'+chosen_t+'_takens_.pkl')
    pd.DataFrame(eigv).to_pickle(PATH2 + 'eigenvectors_heat_matrix_t_'+chosen_t+ '_takens_.pkl')
    print(f'Shape of eigenvectors: {eigv.shape}')
else:
    PATH2 = PATH1+'t_{}/'.format(chosen_t)
    print(f'Loading L from:{PATH2}')   
    L = pd.read_pickle(PATH2+'laplacian_heat_matrix_t_'+chosen_t+'_.pkl').values

    print(f'Loading D from:{PATH2}')   
    D = pd.read_pickle(PATH2+'diagonal_heat_matrix_t_'+chosen_t+'_.pkl').values

    print(f'Computing {NUM_EIGENVALUES} eigenvalues:')
    w, eigv = linalg.eigs(L, k=NUM_EIGENVALUES, M=D, which='SM')

    print('Saving eigendecomposition:')
    pd.DataFrame(w).to_pickle(PATH2 + 'eigenvalues_heat_matrix_t_'+chosen_t+'_.pkl')
    pd.DataFrame(eigv).to_pickle(PATH2 + 'eigenvectors_heat_matrix_t_'+chosen_t+ '_.pkl')

del L, D, w, eigv

## Create time-series:

Loading eigendecomposition for laplacian eigenmaps and NLSA:

In [ ]:
# HEAT KERNEL:
chosen_t = 'mean'
print(f'Loading eigendecomposition for t={chosen_t} at {PATH1}')

w_mean_nlsa = pd.read_pickle(PATH1 + 't_' + chosen_t +
                             '/takens/eigenvalues_heat_matrix_t_' + chosen_t +
                             '_takens_.pkl')
eigv_mean_nlsa = pd.read_pickle(PATH1 + 't_' + chosen_t +
                                '/takens/eigenvectors_heat_matrix_t_' +
                                chosen_t + '_takens_.pkl')

w_mean = pd.read_pickle(PATH1 + 't_' + chosen_t +
                        '/eigenvalues_heat_matrix_t_' + chosen_t + '_.pkl')
eigv_mean = pd.read_pickle(PATH1 + 't_' + chosen_t +
                           '/eigenvectors_heat_matrix_t_' + chosen_t + '_.pkl')

# SIMPLE KERNEL:
print(f'Loading eigendecomposition for simple kernel at {PATH_SIMPLE}')

w_simple_nlsa = pd.read_pickle(PATH_SIMPLE_TAKENS +
                               'eigenvalues_takens_{}.pkl'.format(PERC_NEIGH))
eigv_simple_nlsa = pd.read_pickle(
    PATH_SIMPLE_TAKENS + 'eigenvectors_takens_{}.pkl'.format(PERC_NEIGH))

w_simple = pd.read_pickle(PATH_SIMPLE +
                          'eigenvalues_{}.pkl'.format(PERC_NEIGH))
eigv_simple = pd.read_pickle(PATH_SIMPLE +
                             'eigenvectors_{}.pkl'.format(PERC_NEIGH))
# Safety check:
assert (eigv_simple_nlsa.shape == eigv_mean_nlsa.shape)
assert (eigv_mean.shape == eigv_simple.shape)

print(f'Shape of eigenvectors for NLSA: {eigv_mean_nlsa.shape}')
print(f'Shape of eigenvectors for eigenmaps: {eigv_mean.shape}')

Invert eigenvector 1 and 0 for raw data, weird bug, eigenvector that is nul is in second position for raw data so need to remove it. 

In [ ]:
# weirdly 0 in position 1 so invert position 0 and 1 in eigenvalues:
if DATA == 'raw':
    w_mean[0][1] = w_mean[0][0]
    w_mean[0][0] = 0
    w_mean_nlsa[0][1] = w_mean_nlsa[0][0]
    w_mean_nlsa[0][0] = 0

    w_simple[0][1] = w_simple[0][0]
    w_simple[0][0] = 0
    w_simple_nlsa[0][1] = w_simple_nlsa[0][0]
    w_simple_nlsa[0][0] = 0

Get time component for time-series:

In [ ]:
%%time
if DATA == 'anomalies':
    df = pd.read_csv(INPUT_DATA + 'anomalies_coefficients.csv', sep=',')
else:
    df = pd.read_csv(INPUT_DATA + 'raw_data_coefficients.csv', sep=',')
time = pd.to_datetime(df['Date'])
del df

Create time-series from Laplacian eigenmaps:

In [ ]:
eigv_time_mean = pd.concat([time, pd.DataFrame(eigv_mean)], axis=1)
eigv_time_mean = eigv_time_mean.set_index('Date')
eigv_time_simple = pd.concat([time, pd.DataFrame(eigv_simple)], axis=1)
eigv_time_simple = eigv_time_simple.set_index('Date')

# drop eigenvector that is null:
if DATA == 'raw':
    eigv_time_mean = eigv_time_mean.drop([1], axis=1)
    eigv_time_simple = eigv_time_simple.drop([1], axis=1)
else:
    eigv_time_mean = eigv_time_mean.drop([0], axis=1)
    eigv_time_simple = eigv_time_simple.drop([0], axis=1)

eigv_time_mean.columns = range(1, NUM_EIGENVALUES)
eigv_time_simple.columns = range(1, NUM_EIGENVALUES)

Create time-series for NLSA eigenmaps:

In [ ]:
first_pos_winters = np.load(PATH1 + 'first_pos_winters.npy')
last_pos_winters = np.load(PATH1 + 'last_pos_winters.npy')

indices_of_points = []
for i in range(len(first_pos_winters)):
    indices_of_points.append(
        range(first_pos_winters[i] + TAU, last_pos_winters[i] + 1, 1))
for i in indices_of_points:
    assert (i[-1] - i[0] > 600)

In [ ]:
time_nlsa = time[indices_of_points[0]]

for i in range(1, len(indices_of_points)):
    time_nlsa = pd.concat([time_nlsa, time[indices_of_points[i]]], axis=0)

assert (len(time_nlsa) == eigv_mean_nlsa.shape[0])

time_nlsa = pd.to_datetime(time_nlsa)
time_nlsa.to_csv(PATH_TAKENS + 'time_nlsa.csv')

In [ ]:
eigv_time_mean_nlsa = pd.DataFrame(eigv_mean_nlsa)
eigv_time_mean_nlsa.index = time_nlsa
eigv_time_simple_nlsa = pd.DataFrame(eigv_simple_nlsa)
eigv_time_simple_nlsa.index = time_nlsa

if DATA == 'raw':
    eigv_time_simple_nlsa = eigv_time_simple_nlsa.drop([1], axis=1)
    eigv_time_mean_nlsa = eigv_time_mean_nlsa.drop([1], axis=1)
else:
    eigv_time_simple_nlsa = eigv_time_simple_nlsa.drop([0], axis=1)
    eigv_time_mean_nlsa = eigv_time_mean_nlsa.drop([0], axis=1)

eigv_time_simple_nlsa.columns = range(1, NUM_EIGENVALUES)
eigv_time_mean_nlsa.columns = range(1, NUM_EIGENVALUES)

In [ ]:
winters = {
    1998: pd.date_range('1998-12', '1999-04'),
    2008: pd.date_range('2008-12', '2009-04'),
    2010: pd.date_range('2010-12', '2011-04')
}
for i in range(1, 6):
    if DATA == 'raw':
        if USE_TAKENS:
            np.save(
                f'../../../data/vandermeer/pickles/raw/10perc/t_mean/takens/eignpy_{i}_takens_.npy',
                eigv_time_mean_nlsa[i])
            for year in winters:
                np.save(
                    f'../../../data/vandermeer/pickles/raw/10perc/t_mean/takens/winter_eignpy_{i}_takens_{year}.npy',
                    eigv_time_mean_nlsa[i][winters[year]])
        else:
            np.save(PATH1 + f't_mean/eignpy_{i}.npy', eigv_time_mean[i])
            for year in winters:
                np.save(PATH1 + f't_mean/winter_eignpy_{i}_{year}.npy',
                        eigv_time_mean[i][winters[year]])
    else:
        if USE_TAKENS:
            np.save(
                f'../../../data/vandermeer/pickles/anomalies/10perc/t_mean/takens/eignpy_{i}_takens_an.npy',
                eigv_time_mean_nlsa[i])
            for year in winters:
                np.save(
                    f'../../../data/vandermeer/pickles/anomalies/10perc/t_mean/takens/winter_eignpy_{i}_takens_an_{year}.npy',
                    eigv_time_mean_nlsa[i][winters[year]])
        else:
            np.save(PATH1 + f't_mean/eignpy_{i}_an.npy', eigv_time_mean[i])
            for year in winters:
                np.save(PATH1 + f't_mean/winter_eignpy_{i}_an_{year}.npy',
                        eigv_time_mean[i][winters[year]])

Somehow because of the range of indices I took to create the time-series, the first day of october is also added, which is wrong, so we remove it this way:

In [ ]:
years = range(BEGIN_YEAR, END_YEAR)
years = [str(y) for y in years]
for y in years[1:]:
    i = eigv_time_simple_nlsa[f'{y}-10'].index
    j = eigv_time_mean_nlsa[f'{y}-10'].index
    eigv_time_simple_nlsa.drop(i, inplace = True)
    eigv_time_mean_nlsa.drop(j, inplace = True)

## Plot time-series:

### Special SSW trajectories:

In [ ]:
years = range(BEGIN_YEAR, END_YEAR)
years = [str(y) for y in years]

In [ ]:
from dateutil.relativedelta import relativedelta

dates = [['1980-02-29-12:00:00', 'D'],
         ['1981-03-04-12:00:00', 'D'], ['1981-12-04-12:00:00', 'D'],
         ['1984-02-24-12:00:00', 'D'], ['1985-01-01-12:00:00', 'S'],
         ['1987-01-23-12:00:00', 'D'], ['1987-12-08-12:00:00', 'S'],
         ['1988-03-14-12:00:00', 'S'], ['1989-02-21-12:00:00', 'S'],
         ['1998-12-15-12:00:00', 'D'], ['1999-02-26-12:00:00', 'S'],
         ['2000-03-20-12:00:00', 'D'], ['2001-02-11-12:00:00', 'S'],
         ['2001-12-30-12:00:00', 'D'], ['2003-01-18-12:00:00', 'S'],
         ['2004-01-05-12:00:00', 'D'], ['2006-01-21-12:00:00', 'D'],
         ['2007-02-24-12:00:00', 'D'], ['2008-02-22-12:00:00', 'D'],
         ['2009-01-24-12:00:00', 'S'], ['2010-02-09-12:00:00', 'S'],
         ['2013-01-07-12:00:00', 'S']]

dates_ = []
for el in dates:
    year = str(pd.to_datetime(el[0]).year)
    two_months = [
        str(pd.to_datetime(el[0]) + relativedelta(months=2)), el[1] + '_TM'
    ]
    dates_.append(el)
    if (pd.to_datetime(el[0]) + relativedelta(months=2)).month < 4 and (
            pd.to_datetime(el[0]) + relativedelta(months=2)).month >= 1:
        dates_.append(two_months)
    if (pd.to_datetime(el[0]) + relativedelta(months=2)).month >= 10:
        dates_.append(two_months)

In [ ]:
years_in_d = {}
for el in dates_:
    year = str(pd.to_datetime(el[0]).year)
    if year not in years_in_d:
        years_in_d[year] = [[el[0], el[1]]]
    else:
        l = [j for j in years_in_d[year]]
        l.append([el[0], el[1]])
        years_in_d[year] = l
years_in_d

In [ ]:
# two month trajectories:
trajectories = []
for el in dates:
    year = str(pd.to_datetime(el[0]).year)
    two_months = [str(pd.to_datetime(el[0]) + relativedelta(months=2)), el[1] + '_TM']
    
    if pd.to_datetime(el[0]).month < 4:
        if (pd.to_datetime(el[0]) + relativedelta(months=2)).month < 4:
            trajectories.append([el, two_months])
        else:
            end_of_month = [f'{year}-03-31-12:00:00', el[1] + '_TM']
            trajectories.append([el, end_of_month])           
    else: 
        trajectories.append([el, two_months])
        
traj = {}
for el in trajectories:
    start = el[0]
    end = el[1]
    year = str(pd.to_datetime(start[0]).year)
    range_ = pd.date_range(str(pd.to_datetime(start[0]).year) + '-' +
                               str(pd.to_datetime(start[0]).month) + '-' +
                               str(pd.to_datetime(start[0]).day),
                               str(pd.to_datetime(end[0]).year) + '-' +
                               str(pd.to_datetime(end[0]).month) + '-' +
                               str(pd.to_datetime(end[0]).day),
                               freq='6H')
    type_ = el[0][1]
    if year not in traj:
        traj[year] = [[range_, type_]]
    else:
        l = [j for j in traj[year]]
        l.append([range_, type_])
        traj[year] = l
traj

In [ ]:
# traj -30 days + 10 days:
trajectories_30_10 = []

for el in dates:
    year = str(pd.to_datetime(el[0]).year)
    ten_days = [str(pd.to_datetime(el[0]) + relativedelta(days=10)), el[1] + '_TM']
    one_month = [str(pd.to_datetime(el[0]) - relativedelta(days=30)), el[1]]
    
    # in case we're in jan-april:
    if pd.to_datetime(el[0]).month < 4:
        if (pd.to_datetime(el[0]) + relativedelta(days=10)).month < 4:
            trajectories_30_10.append([one_month, ten_days])
        else:
            end_of_month = [f'{year}-03-31-12:00:00', el[1] + '_TM']
            trajectories_30_10.append([one_month, end_of_month])           
    
    # in case we're in oct-dec:
    else: 
        if (pd.to_datetime(el[0]) - relativedelta(days=30)).month >=12: 
            trajectories_30_10.append([one_month, ten_days])
        else:
            start_of_month = [f'{year}-12-01-12:00:00', el[1]]
            trajectories_30_10.append([start_of_month, ten_days])
traj_30_10 = {}
for el in trajectories_30_10:
    start = el[0]
    end = el[1]
    year = str(pd.to_datetime(start[0]).year)
    range_ = pd.date_range(str(pd.to_datetime(start[0]).year) + '-' +
                               str(pd.to_datetime(start[0]).month) + '-' +
                               str(pd.to_datetime(start[0]).day),
                               str(pd.to_datetime(end[0]).year) + '-' +
                               str(pd.to_datetime(end[0]).month) + '-' +
                               str(pd.to_datetime(end[0]).day),
                               freq='6H')
    type_ = el[0][1]
    if year not in traj_30_10:
        traj_30_10[year] = [[range_, type_]]
    else:
        l = [j for j in traj_30_10[year]]
        l.append([range_, type_])
        traj_30_10[year] = l
traj_30_10

In [ ]:
colors_points = {
    'D': 'orangered',
    'D_TM': 'darkred',
    'S': 'deepskyblue',
    'S_TM': 'darkblue'
}

In [ ]:
def Diff(li1, li2):
    return (list(list(set(li1) - set(li2)) + list(set(li2) - set(li1))))

winters_S, winters_D, winters_ssw = [], [], []
for d in dates:
    year = pd.to_datetime(d[0]).year
    month = pd.to_datetime(d[0]).month
    type_ = d[1]
    if month < 10 and year not in winters_ssw:
        winters_ssw.append(year - 1)
        if type_ == 'S':
            winters_S.append(year - 1)
        else:
            winters_D.append(year - 1)
    if month >= 10 and year not in winters_ssw:
        winters_ssw.append(year)
        if type_ == 'S':
            winters_S.append(year)
        else:
            winters_D.append(year)
            
winters_ssw = np.array(winters_ssw)
winters_S = np.array(winters_S)
winters_D = np.array(winters_D)

winters_no_ssw = np.sort(Diff(range(1979, 2017, 1), winters_ssw))
print('Winters with SSW:')
print(winters_ssw)
print('Winters with SSW type split:')
print(winters_S)
print('Winters with SSW type displacement:')
print(winters_D)
print('Winters without SSW:')
print(winters_no_ssw)

### Plot eigenvalues:

In [ ]:
def plot_eigenvalues(w_mean, w_simple, w_mean_nlsa, w_simple_nlsa,
                     NUM_EIGENVALUES):
    fig, axs = plt.subplots(1, figsize=(10, 5))
    axs.scatter(x=range(0, NUM_EIGENVALUES),
                y=w_mean_nlsa,
                marker='o',
                alpha=0.7,
                label='heat kernel NLSA')
        
    axs.scatter(x=range(0, NUM_EIGENVALUES),
                y=w_mean,
                marker='o',
                alpha=0.7,
                label='heat kernel')
    axs.set_ylabel('Eigenvalues')
    axs.set_xlabel('Number of eigenvalue')
    plt.locator_params(axis='x', nbins=21)
    axs.legend()

In [ ]:
plot_eigenvalues(w_mean, w_simple, w_mean_nlsa, w_simple_nlsa, NUM_EIGENVALUES)

### Clustering:

In [ ]:
NUM_CLUSTERS = 2

In [ ]:
%%time
from sklearn.cluster import SpectralClustering

# Cluster on january-february only:
jan_feb = pd.DataFrame()

y = years[1]
range_ = pd.date_range(f'{y}-01-01', f'{y}-03-01', freq='6H')
jan_feb = eigv_time_mean_nlsa.loc[range_]

for y in years[2:]:
    range_ = pd.date_range(f'{y}-01-01', f'{y}-03-01', freq='6H')
    jan_feb = pd.concat([jan_feb, eigv_time_mean_nlsa.loc[range_]], axis=0)

# take first five eigenvectors
df_ = jan_feb[[1, 2, 3, 4, 5]]
df = np.real(df_.values)
spectral = SpectralClustering(n_clusters=NUM_CLUSTERS, random_state=0,
                              affinity='rbf').fit(df)

jan_feb['cluster_label_spectral_kernel'] = spectral.labels_

In [ ]:
"""
from sklearn.cluster import KMeans

df = np.real(eigv_time_mean_nlsa.values)
kmeans = KMeans(n_clusters=3, random_state=0).fit(df)

eigv_time_mean_nlsa['cluster_label'] = kmeans.labels_"""

In [ ]:
%%time
from sklearn.cluster import SpectralClustering

df = np.real(eigv_time_mean_nlsa.values)
spectral = SpectralClustering(n_clusters=NUM_CLUSTERS, random_state=0,
                              affinity='rbf').fit(df)

eigv_time_mean_nlsa['cluster_label_spectral_kernel'] = spectral.labels_

In [ ]:
PATH1

In [ ]:
"""
from tslearn.clustering import KernelKMeans
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance


# take first five eigenvectors
df_ = jan_feb[[1, 2, 3, 4, 5]]
df = np.real(df_.values)

seed = 0
np.random.seed(seed)
gak_km = KernelKMeans(n_clusters=NUM_CLUSTERS,
                      kernel="gak",
                      kernel_params={"sigma": "auto"},
                      n_init=1,
                      verbose=True,
                      random_state=seed)
y_pred = gak_km.fit_predict(df)
eigv_time_mean_nlsa['cluster_label_kernel'] = y_pred.labels_"""

### Plot all eigenvectors for all years:

In [ ]:
def plot_all_eig_clusters(eigv_time, jan_feb, clusters = False, traj = traj):
    fig, axs = plt.subplots(10, 1, figsize=(20, 25))
    m = 1
    for i in range(10):
        
        full_winters = pd.date_range('1980-01-01', '2017-12-31', freq = '6H')
        df = pd.DataFrame(index=full_winters,
                          data={
                              m:
                              eigv_time_mean_nlsa[m],
                              'cluster_label_spectral_kernel':
                              eigv_time_mean_nlsa['cluster_label_spectral_kernel']
                          })        
        axs[i].plot(df[m], alpha=0.5, color = 'grey')
        colors = {0: 'blue', 1: 'red', 2: 'green'}
        
        if i%2 == 0:
            # All clusters:
            draw_clusters = df.dropna()
            labels = jan_feb['cluster_label_spectral_kernel']
            axs[i].scatter(jan_feb[m].index,
                                       jan_feb[m].values,marker=".",s = 6,
                                       c=labels.apply(lambda x: colors[x]))
        
            axs[i].set_ylabel(f"eigenvector {m}")
            xticks = pd.date_range('1979', '2021', freq='1Y')
            xticks_ = [i.year+1 for i in xticks]
            axs[i].set_xticks(xticks)
            axs[i].set_xticklabels(xticks_)
            axs[i].set_xlim([pd.to_datetime('1979'), pd.to_datetime('2019')])
        
        # Clusters for traj:
        else:
            for year in traj:
                for el in traj[str(year)]:
                    if el[0].month[0] < 4:
                        if el[0].month[-1] < 3:
                            range_ = el[0]
                        else: 
                            y = el[0][0].year
                            month = el[0][0].month
                            d = el[0][0].day
                            range_ = pd.date_range(f'{y}-{month}-{d}', f'{y}-03-01', freq = '6H')
                        trajectory = jan_feb[m][range_]
                        type_ = el[1]
                        labels = jan_feb.loc[trajectory.index]['cluster_label_spectral_kernel']
                        axs[i].scatter(trajectory.index,
                                           trajectory.values,marker=".", s = 6,
                                           c=labels.apply(lambda x: colors[x]))

            axs[i].set_ylabel(f"eigenvector {m}")
            xticks = pd.date_range('1979', '2021', freq='1Y')
            xticks_ = [i.year+1 for i in xticks]
            axs[i].set_xticks(xticks)
            axs[i].set_xticklabels(xticks_)
            axs[i].set_xlim([pd.to_datetime('1979'), pd.to_datetime('2019')])
            m += 1

In [ ]:
colors_lines ={'S':'darkblue', 'D':'darkred', 'D+S':'darkorange', 'N':'green'}

plot_all_eig_clusters(eigv_time_mean_nlsa,jan_feb, traj = traj_30_10)

In [ ]:
def plot_all_eig(eigv_time, colors_lines, traj = traj):
    fig, axs = plt.subplots(10, 1, figsize=(25, 25))
    m = 1
    for i in range(10):
        
        
        full_winers = pd.date_range('1980-01-01', '2017-12-31', freq = '6H')
        df = pd.DataFrame(index=full_winers,
                          data={
                              m:
                              eigv_time_mean_nlsa[m],
                              'cluster_label_spectral_kernel':
                              eigv_time_mean_nlsa['cluster_label_spectral_kernel']
                          })        
        axs[i].plot(df[m], alpha=0.5, color = 'grey')
        colors = ['blue', 'red','green']
        
        for year in traj:
            for el in traj[str(year)]:
                trajectory = eigv_time[m][el[0]]
                type_ = el[1]
                labels = eigv_time.loc[trajectory.index]['cluster_label_spectral_kernel']

                axs[i].plot(trajectory.index,trajectory.values,
                                    alpha=1,color = colors_lines[type_],
                                    linewidth=0.8)
        axs[i].set_ylabel(f"eigenvector {m}")
        xticks = pd.date_range('1979', '2021', freq='1Y')
        xticks_ = [i.year+1 for i in xticks]
        axs[i].set_xticks(xticks)
        axs[i].set_xticklabels(xticks_)
        axs[i].set_xlim([pd.to_datetime('1979'), pd.to_datetime('2019')])
        m += 1

In [ ]:
plot_all_eig(eigv_time_mean_nlsa, colors_lines, traj = traj_30_10)

### Average trajectories with and without SSW:

In [ ]:
def Diff(li1, li2):
    return (list(list(set(li1) - set(li2)) + list(set(li2) - set(li1))))

winters_S, winters_D, winters_ssw = [], [], []
for d in dates:
    year = pd.to_datetime(d[0]).year
    month = pd.to_datetime(d[0]).month
    type_ = d[1]
    if month < 10 and year not in winters_ssw:
        winters_ssw.append(year - 1)
        if type_ == 'S':
            winters_S.append(year - 1)
        else:
            winters_D.append(year - 1)
    if month >= 10 and year not in winters_ssw:
        winters_ssw.append(year)
        if type_ == 'S':
            winters_S.append(year)
        else:
            winters_D.append(year)
            
winters_ssw = np.unique(np.array(winters_ssw))
winters_S = np.unique(np.array(winters_S))
winters_D = np.unique(np.array(winters_D))

winters_no_ssw = np.sort(Diff(range(1979, 2017, 1), winters_ssw))
print('Winters with SSW:')
print(winters_ssw)
print('Winters with SSW type split:')
print(winters_S)
print('Winters with SSW type displacement:')
print(winters_D)
print('Winters without SSW:')
print(winters_no_ssw)

In [ ]:
def average_traj(eig, winters_years):
    # first winter:
    year = winters_years[0]
    winter_of_that_year = pd.date_range(str(year) + '-12',
                                        str(year + 1) + '-04',
                                        freq='6H')
    winter_eig = eig[winter_of_that_year][:485]

    df = pd.DataFrame(index=transform_date(winter_eig.index)[:485],
                      data={year: winter_eig.values})
    for year in winters_years[1:]:
        winter_of_that_year = pd.date_range(str(year) + '-12',
                                            str(year + 1) + '-04',
                                            freq='6H')
        val = eig[winter_of_that_year][:485].values

        df[year] = eig[winter_of_that_year][:485].values

    return np.mean(df, axis=1), df

In [ ]:
def traj_SSW_spring(type_ = 'S', winters_S = winters_S):
    trajectories = []
    for winter in winters_S:
        if str(winter) in traj_30_10:
            for traj in traj_30_10[str(winter)]:
                month_1 = traj[0].month[0]
                # if trajectory of ssw starts before march 
                if month_1 <= 3 and traj[1] in type_:
                    trajectories.append(traj)
    return trajectories

traj_S_spring = traj_SSW_spring(type_ = 'S', winters_S = winters_S)
traj_D_spring = traj_SSW_spring(type_ = 'D', winters_S = winters_D)
traj_ssw_spring = traj_SSW_spring(type_ = ['D','S'], winters_S = winters_ssw)

traj_no_ssw_spring = []
for winter in winters_no_ssw:
    traj_no_ssw_spring.append([
        pd.date_range(str(winter) + '-01', str(winter) + '-04', freq='6H'), 'N'
    ])

In [ ]:
def traj_SSW_winter(type_ = 'S', winters_S = winters_S):
    trajectories = []
    for winter in winters_S:
        if str(winter) in traj_30_10:
            for traj in traj_30_10[str(winter)]:
                month_1 = traj[0].month[0]
                # if trajectory of ssw starts before march 
                if month_1 > 4 and traj[1] in type_:
                    trajectories.append(traj)
    return trajectories

traj_S_winter = traj_SSW_winter(type_ = 'S', winters_S = winters_S)
traj_D_winter = traj_SSW_winter(type_ = 'D', winters_S = winters_D)
traj_ssw_winter = traj_SSW_winter(type_ = ['D','S'], winters_S = winters_ssw)

traj_no_ssw_winter = []
for winter in winters_no_ssw:
    traj_no_ssw_winter.append([
        pd.date_range(str(winter) + '-12-01', str(winter) + '-12-31', freq='6H'), 'N'
    ])

In [ ]:
def avg_traj_spring(eig, trajectories):
    m = 1
    winter_eig = eig[trajectories[0][0]]
    ind = transform_date(winter_eig.index, special_traj = True)
    test = pd.Series(index = ind, data = winter_eig.values)
    if pd.datetime(1984,2,28) in test.index:
        test.drop(test['1984-02-28'].index, axis = 0, inplace = True)
    df = pd.DataFrame(index = pd.date_range(str(1984) + '-01', str(1984) + '-04',
                                                freq='6H'), data = {1: test})
    n = 2
    for traj in trajectories[1:]: 
        winter_eig = eig[traj[0]]

        ind = transform_date(winter_eig.index, special_traj = True)
        test = pd.Series(index = ind, data = winter_eig.values)
        if pd.datetime(1984,2,28) in test.index:
            test.drop(test['1984-02-28'].index, axis = 0, inplace = True)
        df[n] = test
        n+= 1
    
    return np.mean(df, axis=1), df

In [ ]:
def avg_traj_winter(eig, trajectories):
    m = 1
    winter_eig = eig[trajectories[0][0]]
    ind = transform_date(winter_eig.index, special_traj = True)
    test = pd.Series(index = ind, data = winter_eig.values)

    df = pd.DataFrame(index = pd.date_range(str(1983) + '-12-01', str(1983) + '-12-31',
                                                freq='6H'), data = {1: test})
    n = 2
    for traj in trajectories[1:]: 
        winter_eig = eig[traj[0]]

        ind = transform_date(winter_eig.index, special_traj = True)
        test = pd.Series(index = ind, data = winter_eig.values)
        df[n] = test
        n+= 1
    
    return np.mean(df, axis=1), df

In [ ]:
def transform_date(column, special_traj=False):
    
    if special_traj:
        new_col = []
        month = column[0].month
        for i in column:
            month = i.month
            day = i.day
            if month < 4:
                if month == 2 and day == 29:
                    new_col.append(
                        datetime(1984, month, day - 1, i.hour, i.minute,
                                 i.second))
                else:
                    new_col.append(
                        datetime(1984, month, day, i.hour, i.minute, i.second))
            else:
                new_col.append(
                    datetime(1983, month, day, i.hour, i.minute, i.second))
    else:
        new_col = []
        year = column[0].year
        for i in column:
            month = i.month
            day = i.day
            if i.year > year:
                if month == 2 and day == 29:
                    new_col.append(
                        datetime(1984, month, day - 1, i.hour, i.minute,
                                 i.second))
                else:
                    new_col.append(
                        datetime(1984, month, day, i.hour, i.minute, i.second))
            else:
                new_col.append(
                    datetime(1983, month, day, i.hour, i.minute, i.second))
    return new_col

### Histograms according to clusters:

In [ ]:
def select_years(df, winters):
    winters_ = [str(i) for i in winters]
    df_ = df[winters_[0]]
    for y in winters_[1:]:
        df_ = pd.concat([df_, df[y]], axis=1)
    return df_


df_ssw = select_years(eigv_time_mean_nlsa, winters_ssw)
df_no_ssw = select_years(eigv_time_mean_nlsa, winters_no_ssw)
df_ssw_D = select_years(eigv_time_mean_nlsa, winters_D)
df_ssw_S = select_years(eigv_time_mean_nlsa, winters_S)

In [ ]:
fig, ax = plt.subplots(10, 3, figsize=(30, 30))

m = 1
n = 0
for j in range(10):
    for n in range(3):
        avg_no_ssw, df = average_traj(eigv_time_mean_nlsa[m], winters_no_ssw)
        avg_ssw, df = average_traj(eigv_time_mean_nlsa[m], winters_ssw)
        avg_ssw_D, df = average_traj(eigv_time_mean_nlsa[m], winters_D)
        avg_ssw_S, df = average_traj(eigv_time_mean_nlsa[m], winters_S)
        if n == 0:
            sns.distplot(df_no_ssw[m].values,
                         ax=ax[j, n],
                         label='No-SSW',
                         kde=True,
                         hist=False,color='black')
            sns.distplot(df_ssw[m].values,
                         ax=ax[j, n],
                         label='SSW',
                         kde=True,
                         hist=False)
            sns.distplot(df_ssw_D[m].values,
                         ax=ax[j, n],
                         label='SSW-D',
                         kde=True,
                         hist=False)
            sns.distplot(df_ssw_S[m].values,
                         ax=ax[j, n],
                         label='SSW-S',
                         kde=True,
                         hist=False)
            ax[j, n].set_title(f'Histogram for all winters, Eig {m}')
            ax[j, n].ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
            ax[j, n].legend(loc='upper left')
        
        # Plot clusters for january-february period:
        if n == 2:
            c1 = jan_feb[jan_feb['cluster_label_spectral_kernel'] == 0]
            c2 = jan_feb[jan_feb['cluster_label_spectral_kernel'] == 1]
            c3 = jan_feb[jan_feb['cluster_label_spectral_kernel'] == 2]
            sns.distplot(jan_feb[m].values,
                         ax=ax[j, n],
                         label='all',
                         color='black',
                         kde=True,
                         hist=False)
            sns.distplot(c1[m].values,
                         ax=ax[j, n],
                         label='c1',
                         kde=True,
                         hist=False)
            sns.distplot(c2[m].values,
                         ax=ax[j, n],
                         label='c2',
                         kde=True,
                         hist=False)
            sns.distplot(c3[m].values,
                         ax=ax[j, n],
                         label='c3',
                         kde=True,
                         hist=False)
            ax[j, n].set_title(f'Histogram for clusters on jan-feb Eig {m}')
            ax[j, n].legend(loc='upper left')
            m += 1
        
        if n == 1:
            sns.distplot(avg_no_ssw.values,
                         ax=ax[j, n],
                         label='No-SSW',
                         kde=True,
                         hist=False,color='black')
            sns.distplot(avg_ssw.values,
                         ax=ax[j, n],
                         label='SSW',
                         kde=True,
                         hist=False)
            sns.distplot(avg_ssw_D.values,
                         ax=ax[j, n],
                         label='SSW-D',
                         kde=True,
                         hist=False)
            sns.distplot(avg_ssw_S.values,
                         ax=ax[j, n],
                         label='SSW-S',
                         kde=True,
                         hist=False)
            ax[j, n].set_title(f'Eigenvector {m}')
            ax[j, n].ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
            ax[j, n].set_title(f'Histogram for average winter Eig {m}')
            ax[j, n].legend(loc='upper left')

### Dimension reduction, PCA on one winter: 

In [ ]:
def get_winters(eigv, years):
    pd.DataFrame()
    # get winter dates:
    winter = pd.date_range(years[m] + '-12',
                                   years[m + 1] + '-04',
                                   freq='6H')
    #select time series for that winter:
    winter_eig_mean = eigv[winter]
    winter_eig_simple = eigv[winter]

In [ ]:
get_winters(eigv_time_mean_nlsa[1], years)

### One winter NLSA:

In [ ]:
def plot_one_winter_nlsa(eigv_time, eigv_time_simple, sign_mean, sign_simple,
                         year):
    fig, axs = plt.subplots(4, 5, figsize=(23, 20))
    m = 1
    for i in range(4):
        for j in range(5):
            winter_2008 = pd.date_range(str(year) + '-12',
                                        str(year + 1) + '-04',
                                        freq='6H')
            winter_2008_eig_mean = eigv_time[m][winter_2008]
            winter_2008_eig_simple = eigv_time_simple[m][winter_2008]

            axs[i, j].plot(winter_2008_eig_mean.index,
                           sign_mean[m] * winter_2008_eig_mean.values,
                           label='heat kernel')
            axs[i, j].plot(winter_2008_eig_simple.index,
                           sign_simple[m] * winter_2008_eig_simple.values,
                           label='simple')
            axs[i, j].set_title(f"Eigenvector {m}")
            axs[i, j].set_xticks((str(year) + '-12', str(year + 1) + '-01',
                                  str(year + 1) + '-02', str(year + 1) + '-03',
                                  str(year + 1) + '-04'))
            axs[i, j].set_xticklabels(['Dec', 'Jan', 'Feb', 'Mar', 'Apr'])
            m += 1
    plt.legend()

In [ ]:
# NLSA versus Laplacian Eigenmaps: 
sign_mean = [0, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1]
sign_mean_nlsa = [0, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1]
plot_one_winter_nlsa(eigv_time_mean_nlsa, eigv_time_mean,
                     sign_mean_nlsa, sign_mean, 2008)
plt.legend(['NLSA', 'Eigenmaps'])

### Individual winters: 

Plot each winter for an eigenvector

In [ ]:
def plot_each_winter_nlsa(eigv_time,
                          eigv_time_simple,
                          sign_mean,
                          sign_simple,
                          years,
                          num_eig=1, colors_points = colors_points):
    fig, axs = plt.subplots(10, 4, figsize=(30, 35))
    m = 0
    min_ = np.min(eigv_time[num_eig])
    max_ = np.max(eigv_time[num_eig])
    for i in range(10):
        for j in range(4):
            if m == 39:
                break
            # get winter dates:
            winter = pd.date_range(years[m] + '-12',
                                   years[m + 1] + '-04',
                                   freq='6H')
            # select time series for that winter:
            winter_eig_mean = eigv_time[num_eig][winter]
            winter_eig_simple = eigv_time_simple[num_eig][winter]

            axs[i, j].plot(winter_eig_mean.index,
                           sign_mean * winter_eig_mean.values)
            axs[i, j].plot(winter_eig_simple.index,
                           sign_simple * winter_eig_simple.values)

            # add end of year dates (Dec):
            if years[m] in years_in_d:
                for el in years_in_d[years[m]]:
                    d_ = el[0]
                    if pd.to_datetime(d_).month > 10:
                        axs[i, j].plot(pd.to_datetime(d_),
                                       sign_mean * winter_eig_mean[d_],
                                       'X',
                                       color=colors_points[el[1]])
                        axs[i, j].annotate(d_[:-9],
                                           (pd.to_datetime(d_),
                                            sign_mean * winter_eig_mean[d_]))
            # add beginning of next year dates (Jan-Apr):
            if years[m + 1] in years_in_d:
                for el in years_in_d[years[m + 1]]:
                    d_ = el[0]
                    if pd.to_datetime(d_).month < 4:
                        axs[i, j].plot(pd.to_datetime(d_),
                                       sign_mean * winter_eig_mean[d_],
                                       'X',
                                       color=colors_points[el[1]])
                        axs[i, j].annotate(d_[:-9],
                                           (pd.to_datetime(d_),
                                            sign_mean * winter_eig_mean[d_]))

            axs[i, j].set_title("Winter " + years[m] + '-' + years[m + 1])
            axs[i, j].set_xticks(
                (years[m] + '-12', years[m + 1] + '-01', years[m + 1] + '-02',
                 years[m + 1] + '-03', years[m + 1] + '-04'))
            axs[i, j].set_xticklabels(['Dec', 'Jan', 'Feb', 'Mar', 'Apr'])

            if sign_mean > 0:
                axs[i, j].set_ylim([min_, max_])
            else:
                axs[i, j].set_ylim([-max_, -min_])
            m += 1

In [ ]:
plot_each_winter_nlsa(eigv_time_mean_nlsa, eigv_time_mean, 1,
                      1, years, num_eig = 2)
plt.legend(['NLSA', 'Eigenmaps'])
#plt.suptitle('NLSA winters for first eigenvector')

### Histograms:

In [ ]:
df_ssw = select_years(eigv_time_mean_nlsa, winters_ssw)
df_no_ssw = select_years(eigv_time_mean_nlsa, winters_no_ssw)
df_ssw_D = select_years(eigv_time_mean_nlsa, winters_D)
df_ssw_S = select_years(eigv_time_mean_nlsa, winters_S)

In [ ]:
fig, ax = plt.subplots(10, 3, figsize=(12, 20))

m = 1
n = 0
for j in range(10):
    for n in range(3):
        eig = eigv_time_mean_nlsa[m]
        avg_ssw, df = avg_traj_winter(eig, traj_ssw_winter)
        avg_ssw_D, df = avg_traj_winter(eig, traj_D_winter)
        avg_ssw_S, df = avg_traj_winter(eig, traj_S_winter)
        avg_no_ssw, df = avg_traj_winter(eig, traj_no_ssw_winter)

        avg_ssw_spring, df = avg_traj_spring(eig, traj_ssw_spring)
        avg_ssw_D_spring, df = avg_traj_spring(eig, traj_D_spring)
        avg_ssw_S_spring, df = avg_traj_spring(eig, traj_S_spring)
        avg_no_ssw_spring, df = avg_traj_spring(eig, traj_no_ssw_spring)

        if n == 0:
            sns.distplot(avg_no_ssw.values,
                         ax=ax[j, n],
                         color='black',
                         label='No-SSW',
                         kde=True)
            sns.distplot(avg_ssw.values, ax=ax[j, n], label='SSW', kde=True)
            sns.distplot(avg_ssw_D.values,
                         ax=ax[j, n],
                         label='SSW-D',
                         kde=True)
            sns.distplot(avg_ssw_S.values,
                         ax=ax[j, n],
                         label='SSW-S',
                         kde=True)
            if j == 0:
                ax[j, n].set_title(f'Winter trajectories')
            ax[j, n].set_ylabel(f'Eigenvector {m}')
            ax[j, n].ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
        if n == 1:
            sns.distplot(avg_no_ssw_spring.values,
                         ax=ax[j, n],
                         color='black',
                         label='No-SSW',
                         kde=True)
            sns.distplot(avg_ssw_spring.values,
                         ax=ax[j, n],
                         label='SSW',
                         kde=True)
            sns.distplot(avg_ssw_D_spring.values,
                         ax=ax[j, n],
                         label='SSW-D',
                         kde=True)
            sns.distplot(avg_ssw_S_spring.values,
                         ax=ax[j, n],
                         label='SSW-S',
                         kde=True)
            if j == 0:
                ax[j, n].set_title(f'Spring trajectories')
            ax[j, n].set_ylabel('')
            ax[j, n].ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
        if n == 2:
            """
            ax[j, n].plot(avg_no_ssw.dropna().rolling(window=16).mean(),
                          color='black')
            ax[j, n].plot(avg_ssw.dropna().rolling(window=16).mean(),
                          color='C0')
            ax[j, n].plot(avg_ssw_D.dropna().rolling(window=16).mean(),
                          color='C1')
            ax[j, n].plot(avg_ssw_S.dropna().rolling(window=16).mean(),
                          color='C2')

            ax[j, n].plot(avg_no_ssw_spring.dropna().rolling(window=16).mean(),
                          color='black',
                          label='No-SSW')
            ax[j, n].plot(avg_ssw_spring.dropna().rolling(window=16).mean(),
                          color='C0',
                          label='SSW')
            ax[j, n].plot(avg_ssw_D_spring.dropna().rolling(window=16).mean(),
                          color='C1',
                          label='SSW-D')
            ax[j, n].plot(avg_ssw_S_spring.dropna().rolling(window=16).mean(),
                          color='C2',
                          label='SSW-S')"""
            
            sns.distplot(df_no_ssw[m].values,
                         ax=ax[j, n],
                         label='No-SSW',
                         kde=True,hist = False,color='black')
            sns.distplot(df_ssw[m].values,
                         ax=ax[j, n],
                         label='SSW',
                         kde=True, hist = False)
            sns.distplot(df_ssw_D[m].values,
                         ax=ax[j, n],
                         label='SSW-D',
                         kde=True, hist = False)
            sns.distplot(df_ssw_S[m].values,
                         ax=ax[j, n],
                         label='SSW-S',
                         kde=True, hist = False)
            if j == 0:
                ax[j, n].set_title(f'Whole winters')
            ax[j, n].ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
            ax[j, n].legend(loc='upper left')
            ax[j, n].set_ylabel('')
            
            """
            if j == 0:
                ax[j, n].set_title(f'Average trajectory')
            
            ax[j, n].set_xticks(
                ('1983-12', '1984-01', '1984-02', '1984-03', '1984-04'))
            ax[j, n].set_xticklabels(['Dec', 'Jan', 'Feb', 'Mar', 'Apr'])"""
            m += 1
            ax[j, n].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

### Plot overlapping winters:

In [ ]:
def transform_date(column, special_traj=False):
    
    if special_traj:
        new_col = []
        month = column[0].month
        for i in column:
            month = i.month
            day = i.day
            if month < 4:
                if month == 2 and day == 29:
                    new_col.append(
                        datetime(1984, month, day - 1, i.hour, i.minute,
                                 i.second))
                else:
                    new_col.append(
                        datetime(1984, month, day, i.hour, i.minute, i.second))
            else:
                new_col.append(
                    datetime(1983, month, day, i.hour, i.minute, i.second))
    else:
        new_col = []
        year = column[0].year
        for i in column:
            month = i.month
            day = i.day
            if i.year > year:
                if month == 2 and day == 29:
                    new_col.append(
                        datetime(1984, month, day - 1, i.hour, i.minute,
                                 i.second))
                else:
                    new_col.append(
                        datetime(1984, month, day, i.hour, i.minute, i.second))
            else:
                new_col.append(
                    datetime(1983, month, day, i.hour, i.minute, i.second))
    return new_col

In [ ]:
def transform_d(date, year):
    month = date.month
    day = date.day
    if month == 2 and day == 29:
        return datetime(year, month, day - 1, date.hour, date.minute,
                        date.second)
    else:
        
        return datetime(year, month, day, date.hour, date.minute, date.second)

In [ ]:
def winters_supp(eigv_time,
                 ax,
                 sign,
                 eig,
                 traj,
                 colors_lines,
                 colors,winters_no_ssw, winters_ssw,
                 alpha=0.5,
                 linewidth=1,
                 add_traj=False,
                 y_={
                     2010: 'D',
                     1998: 'D+S',
                     2008: 'S',
                     1990: 'N'
                 }):
    years = eigv_time[eig].index.year.unique()

    n = len(years)
    i = 1
    for year in years[:-1]:
        winter_of_that_year = pd.date_range(str(year) + '-12',
                                            str(year + 1) + '-04',
                                            freq='6H')
        winter_eig = eigv_time[eig][winter_of_that_year]
        ax.plot(transform_date(winter_eig.index),
                sign * winter_eig.values,
                color='grey',
                alpha=alpha,
                linewidth=linewidth)

        """
        # Add dates:
        if str(year) in years_in_d:
            for el in years_in_d[str(year)]:
                d_ = pd.to_datetime(el[0])
                if d_.month >= 12:
                    ax.plot(transform_d(d_, 1983),
                            sign * winter_eig[d_],
                            'x',
                            color=colors[el[1]])
        if str(year + 1) in years_in_d:
            for el in years_in_d[str(year + 1)]:
                d_ = pd.to_datetime(el[0])
                if d_.month < 4:
                    ax.plot(transform_d(d_, 1984),
                            sign * winter_eig[d_],
                            'x',
                            color=colors[el[1]])"""

        if year == 1983:
            ax.set_xticks(
                ('1983-12', '1984-01', '1984-02', '1984-03', '1984-04'))
            ax.set_xticklabels(['Dec', 'Jan', 'Feb', 'Mar', 'Apr'])

        if year in y_:
            ax.plot(transform_date(winter_eig.index),
                    sign * winter_eig.values,
                    color=colors_lines[y_[year]],
                    alpha=0.8,
                    linewidth=linewidth,
                    label=f'{year}-{year+1}: {y_[year]}')
        if add_traj:
            if str(year) in traj:
                for el in traj[str(year)]:
                    trajectory = eigv_time[eig][el[0]]
                    type_ = el[1]
                    ax.plot(transform_date(trajectory.index,
                                           special_traj=True),
                            sign * trajectory.values,
                            color='purple',
                            alpha=0.6,
                            linewidth=0.8)
        
        i += 1
    avg_no_ssw, df = average_traj(eigv_time[eig], winters_no_ssw)

    eig = eigv_time[eig]
    avg_ssw_winter, df = avg_traj_winter(eig, traj_ssw_winter)
    avg_ssw_D_winter, df = avg_traj_winter(eig, traj_D_winter)
    avg_ssw_S_winter, df = avg_traj_winter(eig, traj_S_winter)
    avg_no_ssw_winter, df = avg_traj_winter(eig, traj_no_ssw_winter)
    
    avg_ssw_spring, df = avg_traj_spring(eig, traj_ssw_spring)
    avg_ssw_D_spring, df = avg_traj_spring(eig, traj_D_spring)
    avg_ssw_S_spring, df = avg_traj_spring(eig, traj_S_spring)
    avg_no_ssw_spring, df = avg_traj_spring(eig, traj_no_ssw_spring)
    """
    ax.plot(avg_no_ssw_winter.index,
                sign * avg_no_ssw_winter.values,
                linewidth=1, color = 'black', label = 'NO-SSW')
    ax.plot(avg_no_ssw_spring.index,
                sign * avg_no_ssw_spring.values,
                linewidth=1, color = 'black')
    ax.plot(avg_ssw_winter.index,
                sign * avg_ssw_winter.values,
                linewidth=1, color = colors_points['SSW'], label = 'SSW')
    ax.plot(avg_ssw_spring.index,
                sign * avg_ssw_spring.values,
                linewidth=1, color = colors_points['SSW'])
    ax.plot(avg_ssw_D_winter.index,
                sign * avg_ssw_D_winter.values,
                linewidth=1, color = colors_points['D'], label = 'SSW-D')
    ax.plot(avg_ssw_D_spring.index,
                sign * avg_ssw_D_spring.values,
                linewidth=1, color = colors_points['D'])
    ax.plot(avg_ssw_S_winter.index,
                sign * avg_ssw_S_winter.values,
                linewidth=1, color = colors_points['S'], label = 'SSW-S')
    ax.plot(avg_ssw_S_spring.index,
                sign * avg_ssw_S_spring.values,
                linewidth=1, color = colors_points['S'])"""

    plt.legend(loc='upper left')

In [ ]:
colors_lines ={'S':'darkblue', 'D':'darkred', 'D+S':'darkorange', 'N':'green'}
colors_points = {
    'D': 'purple',
    'D_TM': 'darkred',
    'S': 'deepskyblue',
    'S_TM': 'darkblue', 
    'c1-SSW':'orange', 
    'c2-SSW':'orange',
    'c1-NO-SSW':'limegreen', 
    'c2-NO-SSW':'limegreen',
    'SSW':'red', 
    'NO-SSW': 'darkgreen'
    
}
fig, ax = plt.subplots(1, figsize=(10, 5))
winters_supp(eigv_time_mean_nlsa,
             ax,
             1,
             eig=1,
             traj=traj,
             colors_lines=colors_lines,
             colors=colors_points,
             winters_no_ssw=winters_no_ssw,
             winters_ssw=winters_ssw,
             add_traj=True, y_ = {})
#plt.legend(loc='lower right')
plt.title('Overlapping winters for the first Laplacian eigenmap')

### Multiplot: 

In [ ]:
import matplotlib.collections as mcoll
import matplotlib.path as mpath

def colorline(x,
              y,
              ax,
              z=None,
              cmap=plt.get_cmap('copper'),
              norm=plt.Normalize(0.0, 1.0),
              linewidth=3,
              alpha=1.0):
    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))
    # Special case if a single number:
    if not hasattr(
            z, "__iter__"):  # to check for numerical input -- this is a hack
        z = np.array([z])
    z = np.asarray(z)
    segments = make_segments(x, y)
    lc = mcoll.LineCollection(segments,
                              array=z,
                              cmap=cmap,
                              norm=norm,
                              linewidth=linewidth,
                              alpha=alpha)
    #ax = plt.gca()
    ax.add_collection(lc)
    return lc

def make_segments(x, y):
    """
    Create list of line segments from x and y coordinates, in the correct format
    for LineCollection: an array of the form numlines x (points per line) x 2 (x
    and y) array
    """
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    return segments

In [ ]:
def add_arrow(line, position=None, direction='right', size=15, color=None, num_arrows = 5,linewidth=1, alpha=1):
    """
    add an arrow to a line.

    line:       Line2D object
    position:   x-position of the arrow. If None, mean of xdata is taken
    direction:  'left' or 'right'
    size:       size of the arrow in fontsize points
    color:      if None, line color is taken.
    """
    if color is None:
        color = line.get_color()

    xdata = line.get_xdata()
    ydata = line.get_ydata()

    min_ = xdata.min()
    max_ = xdata.max()
    if position is None:
        positions = np.linspace(min_, max_, num_arrows)
    # find closest index
    start_inds = [np.argmin(np.absolute(xdata - p)) for p in positions]
    if direction == 'right':
        end_inds = [start_ind + 1 for start_ind in start_inds]
    else:
        end_inds = [start_ind - 1 for start_ind in start_inds]
    for i in range(len(start_inds)):
        if start_inds[i] < len(xdata) and end_inds[i] < len(xdata): 
            line.axes.annotate('',
                               xytext=(xdata[start_inds[i]], ydata[start_inds[i]]),
                               xy=(xdata[end_inds[i]], ydata[end_inds[i]]),
                               arrowprops=dict(arrowstyle="->", color=color, linewidth=linewidth, alpha=alpha),
                               size=size)

In [ ]:
eig_1 = eigv_time_mean[1]
eig_2 = eigv_time_mean[2]
eig_3 = eigv_time_mean[3]
eig_4 = eigv_time_mean[4]
eig_5 = eigv_time_mean[5]

eig_1_nlsa = eigv_time_mean_nlsa[1]
eig_2_nlsa = eigv_time_mean_nlsa[2]
eig_3_nlsa = eigv_time_mean_nlsa[3]
eig_4_nlsa = eigv_time_mean_nlsa[4]
eig_5_nlsa = eigv_time_mean_nlsa[5]

Save eigenvectors for geographical plots in other notebook:

In [ ]:
plots = [[0], [eig_1, eig_2], [eig_1, eig_3], [eig_1, eig_4], [eig_1, eig_5],
         [eig_2, eig_1], [0], [eig_2, eig_3], [eig_2, eig_4], [eig_2, eig_5],
         [eig_3, eig_1], [eig_3, eig_2], [0], [eig_3, eig_4], [eig_3, eig_5],
         [eig_4, eig_1], [eig_4, eig_2], [eig_4, eig_3], [0], [eig_4, eig_5],
         [eig_5, eig_1], [eig_5, eig_2], [eig_5, eig_3], [eig_5, eig_4], [0]]

plots_nlsa = [[0], [eig_1_nlsa, eig_2_nlsa], [eig_1_nlsa, eig_3_nlsa],
              [eig_1_nlsa, eig_4_nlsa], [eig_1_nlsa, eig_5_nlsa],
              [eig_2_nlsa, eig_1_nlsa], [0], [eig_2_nlsa, eig_3_nlsa],
              [eig_2_nlsa, eig_4_nlsa], [eig_2_nlsa, eig_5_nlsa],
              [eig_3_nlsa, eig_1_nlsa], [eig_3_nlsa, eig_2_nlsa], [0],
              [eig_3_nlsa, eig_4_nlsa], [eig_3_nlsa, eig_5_nlsa],
              [eig_4_nlsa, eig_1_nlsa], [eig_4_nlsa, eig_2_nlsa],
              [eig_4_nlsa, eig_3_nlsa], [0], [eig_4_nlsa, eig_5_nlsa],
              [eig_5_nlsa, eig_1_nlsa], [eig_5_nlsa, eig_2_nlsa],
              [eig_5_nlsa, eig_3_nlsa], [eig_5_nlsa, eig_4_nlsa], [0]]

#### Multiplot with average trajectories:

In [ ]:
def multiplot(eigv_time,
              plots,
              dates_,
              traj,
              colors_lines,
              colors_points,
              winters_no_ssw=winters_no_ssw,
              winters_ssw=winters_ssw):
    fig, axs = plt.subplots(5, 5, figsize=(20, 20))
    m = 0
    y_= {}
    for i in range(5):
        for j in range(5):
            pl = plots[m]
            if i != j:
                #plot grey lines:
                axs[i, j].plot(pl[0],
                               pl[1],
                               alpha=0.8,
                               color='grey',
                               linewidth=0.15)

                # add special trajectories:
                for y in y_:
                    winter_year = pd.date_range(f'{y}-12', f'{y+1}-04')
                    winter_year_eig_0 = pl[0][winter_year]
                    winter_year_eig_1 = pl[1][winter_year]
                    line = axs[i, j].plot(winter_year_eig_0,
                                          winter_year_eig_1,
                                          linewidth=1,
                                          color=colors_lines[y_[y]],
                                          label=f'{y}-{y+1}: {y_[y]}')[0]
                    add_arrow(line)
                
                # add average ssw/no ssw trajectories:
                eig1 = pl[0]
                avg_ssw_winter_1, df = avg_traj_winter(eig1, traj_ssw_winter)
                avg_ssw_D_winter_1, df = avg_traj_winter(eig1, traj_D_winter)
                avg_ssw_S_winter_1, df = avg_traj_winter(eig1, traj_S_winter)
                avg_no_ssw_winter_1, df = avg_traj_winter(eig1, traj_no_ssw_winter)

                avg_ssw_spring_1, df = avg_traj_spring(eig1, traj_ssw_spring)
                avg_ssw_D_spring_1, df = avg_traj_spring(eig1, traj_D_spring)
                avg_ssw_S_spring_1, df = avg_traj_spring(eig1, traj_S_spring)
                avg_no_ssw_spring_1, df = avg_traj_spring(eig1, traj_no_ssw_spring)
                
                eig2 = pl[1]
                avg_ssw_winter_2,df = avg_traj_winter(eig2, traj_ssw_winter)
                avg_ssw_D_winter_2, df = avg_traj_winter(eig2, traj_D_winter)
                avg_ssw_S_winter_2, df = avg_traj_winter(eig2, traj_S_winter)
                avg_no_ssw_winter_2, df = avg_traj_winter(eig2, traj_no_ssw_winter)

                avg_ssw_spring_2, df = avg_traj_spring(eig2, traj_ssw_spring)
                avg_ssw_D_spring_2, df = avg_traj_spring(eig2, traj_D_spring)
                avg_ssw_S_spring_2, df = avg_traj_spring(eig2, traj_S_spring)
                avg_no_ssw_spring_2, df = avg_traj_spring(eig2, traj_no_ssw_spring)
                
                line_no_ssw = axs[i, j].plot(avg_no_ssw_winter_1,
                                          avg_no_ssw_winter_2,
                                          linewidth=1,
                                          color=colors_points['NO-SSW'],
                                          label=f'NO-SSW')[0]
                line_no_ssw = axs[i, j].plot(avg_no_ssw_spring_1,
                                          avg_no_ssw_spring_2,
                                          linewidth=1,
                                          color=colors_points['NO-SSW'])[0]
                
                line_ssw = axs[i, j].plot(avg_ssw_winter_1,
                                          avg_ssw_winter_2,
                                          linewidth=1,
                                          color=colors_points['SSW'],
                                          label=f'SSW')[0]
                
                line_ssw = axs[i, j].plot(avg_ssw_D_winter_1,
                                          avg_ssw_D_winter_2,
                                          linewidth=1,
                                          color=colors_points['D'],
                                          label=f'D')[0]
                
                
                line_ssw = axs[i, j].plot(avg_ssw_S_winter_1,
                                          avg_ssw_S_winter_2,
                                          linewidth=1,
                                          color=colors_points['S'],
                                          label=f'S')[0]
                
                line_ssw = axs[i, j].plot(avg_ssw_S_spring_1,
                                          avg_ssw_S_spring_2,
                                          linewidth=1,
                                          color=colors_points['S'])[0]
                
                line_ssw = axs[i, j].plot(avg_ssw_D_spring_1,
                                          avg_ssw_D_spring_2,
                                          linewidth=1,
                                          color=colors_points['D'])[0]
                
                                
                line_ssw = axs[i, j].plot(avg_ssw_spring_1,
                                          avg_ssw_spring_2,
                                          linewidth=1,
                                          color=colors_points['SSW'])[0] 
                for year in traj:
                    if int(year) not in y_ and int(year) - 1 not in y_:
                        for el in traj[year]:
                            type_ = el[1]
                            winter_year_eig_0 = pl[0][el[0]]
                            winter_year_eig_1 = pl[1][el[0]]
                            line = axs[i, j].plot(winter_year_eig_0,
                                                  winter_year_eig_1,
                                                  linewidth=0.8,
                                                  color='purple',
                                                  alpha=0.2)[0]
                            add_arrow(line,
                                      num_arrows=3,
                                      linewidth=0.8,
                                      alpha=0.2)
                axs[i, j].locator_params(axis='x', nbins=3)
            else:
                # diagonal plots
                winters_supp(eigv_time,
                             axs[i, j],
                             -1,
                             i + 1,
                             alpha=0.4,
                             linewidth=1,
                             traj=traj,
                             colors_lines=colors_lines,
                             winters_no_ssw=winters_no_ssw,
                             winters_ssw=winters_ssw,
                             colors=colors_points,y_ = y_,
                             add_traj=True)
            if i == 0:
                axs[i, j].set_title(j + 1)
            if j == 0:
                axs[i, j].set_ylabel(i + 1)
            if j == 3 and i == 2:
                plt.rc('legend', fontsize=12)
                axs[i, j].legend(loc='lower right')
            m += 1

    plt.show()

In [ ]:
colors_lines ={'S':'darkblue', 'D':'darkred', 'D+S':'darkorange', 'N':'green'}
colors_points = {
    'D': 'green',
    'D_TM': 'darkgreen',
    'S': 'deepskyblue',
    'S_TM': 'darkblue', 
    'SSW':'red', 
    'NO-SSW': 'black'
    
}

multiplot(eigv_time_mean_nlsa, plots_nlsa,dates_,traj,colors_lines, colors_points);

#### Multiplot with all ssw:

In [ ]:
def multiplot_allssw(eigv_time,
              plots,
              dates_,
              traj,
              colors_lines,
              colors_points,
              winters_no_ssw=winters_no_ssw,
              winters_ssw=winters_ssw):
    fig, axs = plt.subplots(5, 5, figsize=(20, 20))
    m = 0
    y_= {}
    for i in range(5):
        for j in range(5):
            pl = plots[m]
            if i != j:
                #plot grey lines:
                axs[i, j].plot(pl[0],
                               pl[1],
                               alpha=0.8,
                               color='grey',
                               linewidth=0.15)
                
                """# add average ssw/no ssw trajectories:
                avg_no_ssw_0,df = average_traj(pl[0], winters_no_ssw)
                avg_no_ssw_1,df = average_traj(pl[1], winters_no_ssw)
                
                avg_ssw_0,df = average_traj(pl[0], winters_ssw)
                avg_ssw_1,df = average_traj(pl[1], winters_ssw)
                
                avg_ssw_0_D,df = average_traj(pl[0], winters_D)
                avg_ssw_1_D,df = average_traj(pl[1], winters_D)
                
                avg_ssw_0_S,df = average_traj(pl[0], winters_S)
                avg_ssw_1_S,df = average_traj(pl[1], winters_S)
                
                line_no_ssw = axs[i, j].plot(avg_no_ssw_0,
                                          avg_no_ssw_1,
                                          linewidth=1,
                                          color=colors_points['NO-SSW'],
                                          label=f'NO-SSW')[0]
                add_arrow(line_no_ssw)
                
                line_ssw = axs[i, j].plot(avg_ssw_0,
                                          avg_ssw_1,
                                          linewidth=1,
                                          color=colors_points['SSW'],
                                          label=f'SSW')[0]
                add_arrow(line_ssw)
                line_ssw = axs[i, j].plot(avg_ssw_0_D,
                                          avg_ssw_1_D,
                                          linewidth=1,
                                          color=colors_points['D'],
                                          label=f'SSW-D')[0]
                add_arrow(line_ssw)
                line_ssw = axs[i, j].plot(avg_ssw_0_S,
                                          avg_ssw_1_S,
                                          linewidth=1,
                                          color=colors_points['S'],
                                          label=f'SSW-S')[0]
                add_arrow(line_ssw)"""
                
                # add trajectories
                for year in traj:
                    if int(year) not in y_ and int(year) - 1 not in y_:
                        for el in traj[year]:
                            #type_ = el[1]
                            type_ = 'D'
                            winter_year_eig_0 = pl[0][el[0]]
                            winter_year_eig_1 = pl[1][el[0]]
                            line = axs[i, j].plot(winter_year_eig_0,
                                                  winter_year_eig_1,
                                                  linewidth=0.8,
                                                  color=colors_points[type_],
                                                  alpha=0.6)[0]
                            add_arrow(line,
                                      num_arrows=3,
                                      linewidth=0.8,
                                      alpha=0.5)
                axs[i, j].locator_params(axis='x', nbins=3)

            else:
                # diagonal plots
                winters_supp(eigv_time,
                             axs[i, j],
                             -1,
                             i + 1,
                             alpha=0.4,
                             linewidth=1,
                             traj=traj,
                             colors_lines=colors_lines,
                             winters_no_ssw=winters_no_ssw,
                             winters_ssw=winters_ssw,
                             colors=colors_points,y_ = y_,
                             add_traj=True)
            if i == 0:
                axs[i, j].set_title(j + 1)
            if j == 0:
                axs[i, j].set_ylabel(i + 1)
            if j == 3 and i == 2:
                plt.rc('legend', fontsize=12)
                axs[i, j].legend(loc='lower right')
            m += 1

    plt.show()

In [ ]:
colors_lines ={'S':'darkblue', 'D':'darkred', 'D+S':'darkorange', 'N':'green'}
colors_points = {
    'D': 'purple',
    'D_TM': 'darkred',
    'S': 'deepskyblue',
    'S_TM': 'darkblue', 
    'SSW':'red', 
    'NO-SSW': 'black'
    
}

multiplot_allssw(eigv_time_mean_nlsa, plots_nlsa,dates_,traj,colors_lines, colors_points);

#### Multiplot with clusters:

In [ ]:
time_series = plots_nlsa[1][0].loc[jan_feb['cluster_label_spectral_kernel'].index]

clusters = jan_feb['cluster_label_spectral_kernel'].loc[time_series.index].values
LABEL_COLOR_MAP = {0 : 'red',
                   1 : 'green',
                   2:'blue',
                   }

label_color = [LABEL_COLOR_MAP[l] for l in clusters]

In [ ]:
def multiplot_clusters(eigv_time,
                       plots,
                       dates_,
                       traj,
                       colors_lines,
                       colors_points,
                       winters_no_ssw=winters_no_ssw,
                       winters_ssw=winters_ssw):
    fig, axs = plt.subplots(5, 5, figsize=(20, 20))
    m = 0
    y_ = {}
    for i in range(5):
        for j in range(5):
            pl = plots[m]
            if i != j:
                #plot grey lines:
                axs[i, j].plot(pl[0],
                               pl[1],
                               alpha=0.8,
                               color='grey',
                               linewidth=0.15)

                time_series1 = pl[0].loc[jan_feb['cluster_label_spectral_kernel'].index]
                time_series2 = pl[1].loc[jan_feb['cluster_label_spectral_kernel'].index]
                clusters = jan_feb['cluster_label_spectral_kernel'].loc[
                    time_series.index].values
                LABEL_COLOR_MAP = {
                    0: 'red',
                    1: 'green',
                    2: 'blue',
                }

                label_color = [LABEL_COLOR_MAP[l] for l in clusters]
                axs[i, j].scatter(time_series1,
                                  time_series2,
                                  alpha=0.3,
                                  c=label_color,
                                  s=2,
                                  linewidth=0.15)

                # add special trajectories:
                for y in y_:
                    winter_year = pd.date_range(f'{y}-12', f'{y+1}-04')
                    winter_year_eig_0 = pl[0][winter_year]
                    winter_year_eig_1 = pl[1][winter_year]
                    line = axs[i, j].plot(winter_year_eig_0,
                                          winter_year_eig_1,
                                          linewidth=1,
                                          color=colors_lines[y_[y]],
                                          label=f'{y}-{y+1}: {y_[y]}')[0]
                    add_arrow(line)

                for year in traj:
                    if int(year) not in y_ and int(year) - 1 not in y_:
                        for el in traj[year]:
                            type_ = el[1]
                            winter_year_eig_0 = pl[0][el[0]]
                            winter_year_eig_1 = pl[1][el[0]]
                            line = axs[i, j].plot(winter_year_eig_0,
                                                  winter_year_eig_1,
                                                  linewidth=0.8,
                                                  color='purple',
                                                  alpha=0.2)[0]
                            add_arrow(line,
                                      num_arrows=3,
                                      linewidth=0.8,
                                      alpha=0.2)
                axs[i, j].locator_params(axis='x', nbins=3)

            else:
                # diagonal plots
                winters_supp(eigv_time,
                             axs[i, j],
                             -1,
                             i + 1,
                             alpha=0.4,
                             linewidth=1,
                             traj=traj,
                             colors_lines=colors_lines,
                             winters_no_ssw=winters_no_ssw,
                             winters_ssw=winters_ssw,
                             colors=colors_points,
                             y_=y_,
                             add_traj=False)
            if i == 0:
                axs[i, j].set_title(j + 1)
            if j == 0:
                axs[i, j].set_ylabel(i + 1)
            if j == 3 and i == 2:
                plt.rc('legend', fontsize=12)
                axs[i, j].legend(loc='lower right')
            m += 1

    plt.show()

In [ ]:
multiplot_clusters(eigv_time_mean_nlsa, plots_nlsa, dates_, traj, colors_lines,
                   colors_points)

shows that there is some structure, can’t say what without doing the spacial reconstruction, here have nonlinear dependance
look at spatial pattern of third eigenvector of anomalies, circles capture some rotation 
try to understand the states of the vortex and is there a clear classification between those states
read up on lores attractor, represent climatical systems with 3d summaries looking like some buterlfly
taking 1 versus 2 we can define three regions, select times which correspond to theses regions and taking these samples in the original space we compute means if these samples and plot in original space —> give average pattern which corresponds to data
pattern 2 versus 5 looks best to define about three regions

for anomalies, can take just average in quadrants of outer regions for the same kind of plot

for diagonal of anomalies, take threshold and take time steps above threshold and look at what average of those look like

tau  = 1 month (30 days) 

### Amplitude:

In [ ]:
def calculate_ampl(eig_1,eig_2):
    r = np.sqrt(np.real(eig_1.values)**2 + np.real(eig_2.values**2))
    amplitude = pd.Series(r, index = eig_1.index)
    return amplitude

In [ ]:
# plot overlapping amplitudes:
def ampl_supp(eig1,
              eig2,
              ax,
              sign,
              traj,
              colors_lines=colors_lines,
              colors=colors_points,
              alpha=0.5,
              linewidth=1,
              add_traj=False,
              y_={
                  2010: 'D',
                  1998: 'D+S',
                  2008: 'S',
                  1990: 'N'
              }):
    amplitude = calculate_ampl(eig1, eig2)
    years = amplitude.index.year.unique()
    n = len(years)
    i = 1
    for year in years[:-1]:
        winter_of_that_year = pd.date_range(str(year) + '-12',
                                            str(year + 1) + '-04',
                                            freq='6H')
        winter_ampl = amplitude[winter_of_that_year]
        ax.plot(transform_date(winter_ampl.index),
                sign * winter_ampl.values,
                color='grey',
                alpha=alpha,
                linewidth=linewidth)

        # Add dates:
        if str(year) in years_in_d:
            for el in years_in_d[str(year)]:
                d_ = pd.to_datetime(el[0])
                if d_.month >= 12:
                    ax.plot(transform_d(d_, 1983),
                            sign * winter_ampl[d_],
                            'x',
                            color=colors[el[1]])
        if str(year + 1) in years_in_d:
            for el in years_in_d[str(year + 1)]:
                d_ = pd.to_datetime(el[0])
                if d_.month < 4:
                    ax.plot(transform_d(d_, 1984),
                            sign * winter_ampl[d_],
                            'x',
                            color=colors[el[1]])
        # special trajectories:
        if year in y_:
            ax.plot(transform_date(winter_ampl.index),
                    sign * winter_ampl.values,
                    color=colors_lines[y_[year]],
                    alpha=0.8,
                    linewidth=linewidth,
                    label=f'{year}-{year+1}: {y_[year]}')

        if add_traj:
            if str(year) in traj and year not in y_:
                for el in traj[str(year)]:
                    trajectory = amplitude[el[0]]
                    type_ = el[1]
                    ax.plot(transform_date(trajectory.index,
                                           special_traj=True),
                            sign * trajectory.values,
                            color=colors[type_],
                            alpha=0.5,
                            linewidth=0.8)

        if year == 1983:
            ax.set_xticks(
                ('1983-12', '1984-01', '1984-02', '1984-03', '1984-04'))
            ax.set_xticklabels(['Dec', 'Jan', 'Feb', 'Mar', 'Apr'])
        i += 1
    plt.legend(loc='upper left')

In [ ]:
fig, ax = plt.subplots(1)
ampl_supp(eigv_time_mean_nlsa[1],eigv_time_mean_nlsa[2], ax, 1, traj = traj, add_traj =True)

In [ ]:
def multiplot_amplitude(eigv_time, plots, dates_, traj, colors_lines,
                        colors_points):
    fig, axs = plt.subplots(5, 5, figsize=(20, 20))
    m = 0
    for i in range(5):
        for j in range(5):

            #y_= {2010:'D', 1998:'D+S', 2008:'S', 1990:'N'}
            y_ = {1990: 'N'}
            pl = plots[m]
            if j > i:
                amplitude = calculate_ampl(pl[0], pl[1])
                ampl_supp(pl[0],
                          pl[1],
                          axs[i, j],
                          1,
                          alpha=0.4,
                          linewidth=1,
                          colors_lines=colors_lines,
                          colors=colors_points,
                          traj=traj,
                          add_traj=True,
                          y_=y_)
                axs[i, j].locator_params(axis='x', nbins=3)

            if j == i:
                # diagonal plots
                winters_supp(eigv_time,
                             axs[i, j],
                             -1,
                             i + 1,
                             alpha=0.4,
                             linewidth=1,
                             traj=traj,
                             colors_lines=colors_lines,
                             colors=colors_points,
                             add_traj=True,
                             y_=y_)
            if j < i:
                #plot grey lines:
                axs[i, j].plot(pl[0],
                               pl[1],
                               alpha=0.8,
                               color='grey',
                               linewidth=0.15)
                # add special dates:
                for el in dates_[1:]:
                    d_ = el[0]
                    axs[i, j].plot(pl[0][d_],
                                   pl[1][d_],
                                   'x',
                                   color=colors_points[el[1]])
                # add special trajectories:
                for y in y_:
                    winter_year = pd.date_range(f'{y}-12', f'{y+1}-04')
                    winter_year_eig_0 = pl[0][winter_year]
                    winter_year_eig_1 = pl[1][winter_year]
                    line = axs[i, j].plot(winter_year_eig_0,
                                          winter_year_eig_1,
                                          linewidth=1,
                                          color=colors_lines[y_[y]],
                                          label=f'{y}-{y+1}: {y_[y]}')[0]
                    add_arrow(line)
                for year in traj:
                    if int(year) not in y_ and int(year) - 1 not in y_:
                        for el in traj[year]:
                            type_ = el[1]
                            winter_year_eig_0 = pl[0][el[0]]
                            winter_year_eig_1 = pl[1][el[0]]
                            line = axs[i, j].plot(winter_year_eig_0,
                                                  winter_year_eig_1,
                                                  linewidth=0.8,
                                                  color=colors_points[type_],
                                                  alpha=0.7)[0]
                            add_arrow(line,
                                      num_arrows=3,
                                      linewidth=0.7,
                                      alpha=0.8)
                axs[i, j].locator_params(axis='x', nbins=3)
            if i == 0:
                axs[i, j].set_title(j + 1)
            if j == 0:
                axs[i, j].set_ylabel(i + 1)
            m += 1
    plt.show()

In [ ]:
multiplot_amplitude(eigv_time_mean_nlsa, plots_nlsa, dates_, traj, colors_lines,
                    colors_points)

### Power Spectrums:
Power spectrum plot for all years and then see something every 7 months

In [ ]:
sampling_rate_seconds = 4 / (24 * 3600)
print(f'Frequency: {sampling_rate_seconds} [s-1]')
sampling_rate_days = 4
print(f'Frequency: {sampling_rate_days} [d-1]')
sampling_time = 1 / sampling_rate_days
print(f'Sampling every  {sampling_time*24} hours')
year_t = 365
two_year_t = (365 / 2)
three_year_t = (365 / 3)
freq_year = 1 / year_t
freq_two_year = 1 / two_year_t
freq_three_year = 1 / three_year_t

Pperations on frequency : log(frequency) -> in time 1/frequency
indicate the 1/year, 2/year and 3/year frequencie

Estimate power spectral density using a periodogram. Change to 1/frequency to time.

In [ ]:
"""
Power spectrum with zero padding for the missing dates:
"""
def pwr_compl(eigv_time, sampling_rate_days, number, axs):
    full_time = pd.date_range(start='1/1/1979', end='1/1/2019', freq='6H')
    full_df = pd.DataFrame(np.zeros((len(full_time), 20)))
    eigv_time = eigv_time.copy()
    full_df.index = full_time
    full_df.columns = [str(i) for i in eigv_time.columns]
    eigv_time.columns = [str(i) for i in eigv_time.columns]

    cols_to_use = full_df.transpose().columns.difference(
        eigv_time.transpose().columns)
    full_df_time = pd.merge(left=full_df.transpose()[cols_to_use],
                            right=eigv_time.transpose(),
                            how='right',
                            left_index=True,
                            right_index=True,
                            suffixes='_x').transpose()
    full_df_time = full_df_time.sort_index()
    f, Pxx_den = signal.periodogram(full_df_time[str(number)], sampling_rate_days)
    axs.semilogy(np.log(1 / f), Pxx_den)
    axs.axvline(np.log(year_t), color='r', linestyle=':', label='Every year')
    axs.axvline(np.log(two_year_t),
                color='g',
                linestyle=':',
                label='Twice per year')
    axs.axvline(np.log(three_year_t),
                color='orange',
                linestyle=':',
                label='Thrice per year')
    #axs.set_title("Powerspectrum of first eigenvector of Heat kernel")
    axs.set_xticklabels([])
    #axs.set_xlabel('Days')
    #axs.set_ylabel('log(time [d])')

In [ ]:
if DATA == 'raw':
    sign_mean = [0, 1, 1, -1, -1,1,1,1,1,1,1,1, 1, -1, -1,1,1,1,1,1,1]
    sign_simple = [0, 1, 1, -1, -1,-1,1,-1,1,1,1,1, 1, -1, -1,1,1,1,1,1,1]
    if USE_TAKENS:
        sign_mean = [0, 1, 1, -1, -1,1,1,1,1,1,1,1, 1, -1, -1,1,1,1,1,1,1]
        sign_simple = [0, 1, 1, 1, -1,-1,-1,1,-1,1,-1,-1, 1, 1, 1,-1,-1,1,1,1,1]
else:
    sign_mean = [0, 1, 1, -1, -1,1,1,1,1,1,1,1, 1, -1, -1,1,1,1,1,1,1]
    sign_simple = [0, 1, 1, 1, -1,1,1,1,-1,1,1,1, 1, 1, 1,1,1,-1,1,-1,-1]
    if USE_TAKENS:
        sign_mean = [0, 1, 1, -1, -1,1,1,1,1,1,1,1, 1, -1, -1,1,1,1,1,1,1]
        sign_simple = [0, -1, 1, 1, -1,-1,1,1,1,1,-1,1, 1, -1, 1,-1,1,-1,-1,1,1]
eigv_time_mean_nlsa =   eigv_time_mean_nlsa.drop('cluster_label_spectral_kernel', axis = 1) 
fig, axs = plt.subplots(10, 4, figsize=(30, 35))
m = 1
n = 1
for j in range(4):
    for i in range(10):
        if j==0 or j==2:
            winter_2008 = pd.date_range('2008-12', '2009-04')
            winter_2008_eig_mean = eigv_time_mean_nlsa[n][winter_2008]

            axs[i, j].plot(winter_2008_eig_mean.index,
                           sign_mean[n] * winter_2008_eig_mean.values,
                           label='heat kernel')

            axs[i, j].set_xticklabels([])
            axs[i, j].set_ylabel(f"Eigenvector {n}")
            if i==9:
                #axs[i, j].set_xticklabels(['Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Ap'])
                
                axs[i, j].set_xticks(
                ('2008'+ '-12', '2009' + '-01', '2009' + '-02',
                 '2009' + '-03', '2009' + '-04'))
                axs[i, j].set_xticklabels(['Dec', 'Jan', 'Feb', 'Mar', 'Apr'])
                axs[i,j].set_xlabel('Months')
            n += 1
        else:   
            pwr_compl(eigv_time_mean_nlsa, sampling_rate_days, m, axs[i,j])
            m+= 1
            if i==9:
                axs[i,j].set_xticklabels(['0', '1', '7', '54', '403', '2980'])
                axs[i,j].set_xlabel('Days')
plt.legend()

### Projection on geographical data: 
Perform projection $X'\in R^{1001x1}= X^T D \phi_i$. $X'_i = X D\phi_i \phi_i^T, \phi \in R^{33960}, X \in R^{Nx1001}$ 

In [ ]:
%%time
chosen_t = 'mean'
if DATA == 'anomalies':
    if USE_TAKENS:
        print(f'Reading {DATA} data from {INPUT_DATA}')
        df = pd.read_csv(INPUT_DATA + 'anomalies_coefficients.csv', sep=',')
        print(f'Reading eigenvectors from {PATH1}')
        eigv = pd.read_pickle(PATH1 + 't_' + chosen_t +
                              '/takens/eigenvectors_heat_matrix_t_' +
                              chosen_t + '_takens_.pkl').values
        eigv = pd.DataFrame(eigv).drop([0], axis=1).values

    else:
        print(f'Reading {DATA} data from {INPUT_DATA}')
        df = pd.read_csv(INPUT_DATA + 'anomalies_coefficients.csv', sep=',')
        print(f'Reading eigenvectors from {PATH1}')
        eigv = pd.read_pickle(
            PATH1 + 't_mean/eigenvectors_heat_matrix_t_mean_.pkl').values
        eigv = pd.DataFrame(eigv).drop([0], axis=1).values

else:
    if USE_TAKENS:
        print(f'Reading {DATA} data from {INPUT_DATA}')
        df = pd.read_csv(INPUT_DATA + 'raw_data_coefficients.csv', sep=',')
        print(f'Reading eigenvectors from {PATH1}')
        eigv = pd.read_pickle(PATH1 + 't_' + chosen_t +
                              '/takens/eigenvectors_heat_matrix_t_' +
                              chosen_t + '_takens_.pkl').values
        eigv = pd.DataFrame(eigv).drop([1], axis=1).values
    else:
        print(f'Reading {DATA} data from {INPUT_DATA}')
        df = pd.read_csv(INPUT_DATA + 'raw_data_coefficients.csv', sep=',')
        print(f'Reading eigenvectors from {PATH1}')
        eigv = pd.read_pickle(
            PATH1 + 't_mean/eigenvectors_heat_matrix_t_mean_.pkl').values
        eigv = pd.DataFrame(eigv).drop([1], axis=1).values

print(f'Reading diagonal matrix from {PATH1}')
if USE_TAKENS:
    D = pd.read_pickle(PATH1 +
                       't_mean/takens/diagonal_heat_matrix_t_mean_takens_.pkl').values
    X = df.drop(['Unnamed: 0', 'Date'], axis=1)
    X_nlsa = X.loc[indices_of_points[0]]
    for i in range(1, len(last_pos_winters)):
        X_nlsa = pd.concat([X_nlsa, X.loc[indices_of_points[i]]], axis = 0)
    Xt = X_nlsa.values.transpose()
else:
    D = pd.read_pickle(PATH1 +
                       't_mean/diagonal_heat_matrix_t_mean_.pkl').values

    Xt = df.drop(['Unnamed: 0', 'Date'], axis=1).values.transpose()

print(f'Xt shape: {Xt.shape}')
print(f'Diagonal matrix shape: {D.shape}')
print(f'Eigenvectors shape: {eigv.shape}')

Projections on eigenvectors:

In [ ]:
for i in range(NUM_EIGENVALUES-1):
    X_prime_i = Xt @ D @  eigv[:,i]
    np.save(PATH1 +'t_mean/takens/'+ 'X_prime_{}_takens_.npy'.format(i), X_prime_i)
    print('Saving X_prime_{}_takens_.npy at {}'.format(i, PATH1+'t_mean/takens/'))
del Xt, D, eigv

### Comments: 
- First request a lot of eigenvectors and check that only one eigenvector that has eigenvalue close to zero (~$10^{-15}$) and might need to increase number of neighbours otherwise (check if graph is connected): **DONE FOR 50 EIGENVALUES, ONLY ONE ZERO**
- Take the mean over all winters : **DONE**
- Change to heat kernel and compute for different kernel values, and more than 10 eigenvectors. Plot also the eigenvalues. **DONE**
- To choose t: look at distances and take mean or max of all distances among neighbours. Check if t is not the squared of distance. For large t we should have something close to what we have now. 
- Move to anomaly data
- Power spectrum with missing data ? Compute power spectrum for other eigenvectors. So up to 7 months alright and then a gap and the rest should start at 360 and multiply by $365/(7x30)$ **DONE**

- anomaly data estimated seasonality from raw data. estimate smooth mean and remove it to obtain the anomaly. 
- switch from default weight to gaussian kernel, optimise bandwidth computational demanding, try multiple experiments, assess the best bandwidth also define what best means
- apply also to anomaly data to see if consistent
- include to the model so that it can capture the dynamics of system, those things are called backends embedding (use multiple time steps to represent one sample -> sequences) —> apply and optimite previous all to new representation of data 

Pic raw data so that the first mode of variability should be the seasonality 
capturing something, huge variability but don’t know what

gaussian kernel approximates distance on manifold using this laplacian graph. The 0-1 weights is not very realistic because two samples that might be similar they might be different, For now they all have the sae weight. Not very good to approximate distances. For bandwidth t goes to 0 and a lot of data, converge to the distance on the manifold. 
